In [1]:
import pandas as pd
import numpy as np
import cx_Oracle
import os
import json
import datetime
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings(action="ignore")
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import bs4
import requests
import time
import math
import random
from shapely.geometry import Point, Polygon
import time as timesys
from pyproj import Geod
from haversine import haversine
pd.set_option('display.max_columns', None)

In [2]:
the_number_of_segment = 20

# 전항지 출항 확인

전항지 출항 지점 확인

In [3]:
import os
 
path_dir = 'voy_path'
 
file_list = os.listdir(path_dir)
file_new_list = []
for i in file_list:
    if "portmis" in i:
        file_new_list.append(i)

In [4]:
# ATA 확인
def check_ATA(df_test):
    for i in df_test.index:
        LONGITUDE = df_test.loc[i, 'longitude']
        LATITUDE = df_test.loc[i, 'latitude']
        if Point(LONGITUDE, LATITUDE).within(poly_busan):    
            stop_index = i - 1
            break
            continue
    return stop_index

# 시각화 툴로 processed된 것들 확인

항로 클러스터링

In [5]:
# dense_upper_china_list = ['COSCO_FOS_VRKR6_23processed'
# ,'AKA_BHUM_9V7282_7processed'
# , 'COSCO_FOS_VRKR6_25processed'
# , 'MSC_LA_SPEZIA_DIGX2_2processed'
# , 'OOCL_NEW_YORK_VRJS2_2processed'
# , 'MSC_WASHINGTON_5LDP4_4processed'
# , 'MSC_SOFIA_PAZ_D5FL9_2processed'
# , 'ZHU_CHENG_XIN_ZHOU_VRKR6_26processed'
# , 'OOCL_HAMBURG_VRZK9_6processed'
# , 'SEAMAX_STRATFORD_V7EP2_6processed'
# , 'OOCL_LUXEMBOURG_VRGO3_5processed'
# , 'ZHU_CHENG_XIN_ZHOU_VRKR6_25processed'
# , 'MSC_LUCIANA_3FTY3_4processed'
# , 'MSC_RENEE_VRPL2_5processed'
# , 'COSCO_FOS_VRKR6_24processed'
# , 'COSCO_FOS_VRKR6_20processed'
# , 'COSCO_FOS_VRKR6_26processed'
# , 'ZHU_CHENG_XIN_ZHOU_VRKR6_23processed'
# , 'OOCL_HAMBURG_VRZK9_5processed']

# AKA_BHUM_voyages_list = [
#     'COSCO_FOS_VRKR6_2022_20processed', 'COSCO_FOS_VRKR6_2022_23processed',
#     'COSCO_FOS_VRKR6_2022_24processed', 'COSCO_FOS_VRKR6_2022_25processed',
#     'AKA_BHUM_9V7282_2022_7processed', 'COSCO_FOS_VRKR6_2022_27processed',
#     'COSCO_FOS_VRKR6_2022_28processed', 'AKA_BHUM_9V7282_2022_7processed',
#     'COSCO_FOS_VRKR6_2022_30processed'
# ]

AKA_BHUM_voyages_list = [
    'AKA_BHUM_9V7282_2022_7processed', 'AKA_BHUM_9V7282_2022_8processed'
]
OOCL_NEW_YORK_voyages_list = [
    'OOCL_NEW_YORK_VRJS2_2022_1processed',
    'OOCL_NEW_YORK_VRJS2_2022_2processed',
    'OOCL_NEW_YORK_VRJS2_2022_3processed'
]

OOCL_LUXEMBOURG_voyages_list = [
    'OOCL_LUXEMBOURG_VRGO3_2022_5processed',
    'OOCL_LUXEMBOURG_VRGO3_2022_6processed'
]

OOCL_GENOA_voyages_list = [
    'OOCL_GENOA_VROK9_2022_3processed', 'OOCL_GENOA_VROK9_2022_5processed'
]

OOCL_HAMBURG_voyages_list = [
    'OOCL_HAMBURG_VRZK9_2022_5processed', 'OOCL_HAMBURG_VRZK9_2022_6processed'
]

SEAMAX_STRATFORD_voyages_list = [
    'SEAMAX_STRATFORD_V7EP2_2022_6processed',
    'SEAMAX_STRATFORD_V7EP2_2022_7processed'
]

ZIM_CHARLESTORN_voyages_list = [
    'ZIM_CHARLESTON_VRUC3_2022_6processed',
    'ZIM_CHARLESTON_VRUC3_2022_7processed'
]

single_voyage_list = [
    'MSC_ARIANE_3FDA3_2022_1processed', 'MSC_DANIT_3FZU8_2022_2processed',
    'MSC_EVA_3EVM7_2022_1processed', 'MSC_EXPRESS_III_A8LG7_2022_1processed',
    'MSC_KANOKO_D5TM3_2023_1processed', 'MSC_LA_SPEZIA_DIGX2_2022_2processed',
    'MSC_VANESSA_HOWD_2023_1processed', 'MSC_VIRGO_CQEB9_2023_2processed',
    'MSC_SOFIA_PAZ_D5FL9_2022_2processed',
    'MSC_WASHINGTON_5LDP4_2022_4processed',
    'MSC_RAVENNA_A8ZU8_2022_3processed', 'MSC_VANDYA_3FDW4_2022_2processed',
    'MSC_RENEE_VRPL2_2022_5processed', 'SEASPAN_OCEANIA_VRBI2_2023_1processed',
    'SEASPAN_BREEZE_VRNL8_2023_1processed', 'VALENCE_9HA3398_2023_1processed'
]

In [6]:
upper_china_path_dir = 'voy_path/upper_china/processed/'

In [7]:
# 가는 길인데 굳이 오래 돌아갈 필요는 없잖아 알고리즘
def optimal_next_point(current_position, shortest_point_idx, shortest_distance,
                       path_df):
    posi_c = current_position
    latitude_i = path_df.loc[shortest_point_idx, "latitude"]
    longitude_i = path_df.loc[shortest_point_idx, "longitude"]
    posi_i = (latitude_i, longitude_i)
    min_d = shortest_distance
    n = shortest_point_idx + 1
    while True:
        try:
            latitude_n = path_df.loc[n, "latitude"]
            longitude_n = path_df.loc[n, "longitude"]
            posi_n = (latitude_n, longitude_n)
            distance = haversine(posi_c, posi_n, unit="km")
            if n == shortest_point_idx + 1:
                path_distance = haversine(posi_i, posi_n, unit="km")
                paths_distance = path_distance
            else:
                path_distance = haversine(posi_n, posi_previous_n, unit="km")
                paths_distance += path_distance

            if distance < (paths_distance + min_d):
                optimal_next_point_idx = n
                return optimal_next_point_idx
            elif n == path_df.index[-1]:
                print("no better optimal_point")
                return shortest_point_idx + 1
            else:
                n = n + 1
                posi_previous_n = posi_n
        except:
            print("no better optimal_point")
            return shortest_point_idx + 1

In [8]:
# 인덱스 번호를 가지고 항로의 segment 번호를 찾음
# 만약 segment의 마지막 번호라면 seg_no를 하나 더한다.

def find_segment_no(optimal_point_idx, path_df, the_number_of_segment):
    seg_no = optimal_point_idx // (len(path_df) / the_number_of_segment)
    next_no = (optimal_point_idx + 1) // (len(path_df) / the_number_of_segment)
    if seg_no != next_no:
        return int(next_no)
    else:
        return int(seg_no)

In [9]:
# df 내의 포인트 전체 길이 구하는 함수
def reckon_point_lengths(df_test_2):
    distance = 0
    df_test_2.reset_index(inplace=True, drop=True)
    for idx in df_test_2.index:
        if idx != 0:
            LONGITUDE = df_test_2.loc[idx, "longitude"]
            LATITUDE = df_test_2.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = df_test_2.loc[idx - 1, "longitude"]
            LATITUDE = df_test_2.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)

            distance += haversine(current_coord, previous_current_coord, unit="km")
    return distance

In [10]:
def cal_duration(speed, distance):
    hour = int(distance // km_h)
    minute = int(
        (distance / km_h - hour) * 60)
    second = ((distance / km_h - hour) * 60 - minute) * 60
    duration = datetime.timedelta(hours=hour,minutes=minute, seconds=second)
    return duration

# Tianjin to Busan PNIT

## AKA_BHUM

In [11]:
coordinates_array = pd.DataFrame(data={"latitude" : [],"longitude" : []})

for i in AKA_BHUM_voyages_list:
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    coordinates_array = coordinates_array.append(df[["latitude","longitude"]], ignore_index=True)

In [12]:
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
from shapely.geometry import LineString

In [14]:
from scipy import interpolate
import pandas as pd

## below 3 knot 제거

In [15]:
interpolation_number = "1000"

### interpolation 1000

In [16]:
max_speed_list = []
min_speed_list = []
for i in AKA_BHUM_voyages_list:
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    max_speed_list.append(max_speed)
    min_speed_list.append(min_speed)
min_speed = np.min(min_speed_list)
max_speed = np.max(max_speed_list)

# norm = plt.Normalize(min_speed, max_speed)
# cmap = plt.get_cmap('RdYlGn')
# sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
# Create a ScalarMappable with the colormap and normalization

mean_speed_dict = {"voyage" : [],"mean_speed" : []}

for i in AKA_BHUM_voyages_list:
#     fig, ax = plt.subplots()
    
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    # 3노트 이하 데이터 삭제
    data = data[data["speed"] >= 3]
    data.reset_index(inplace=True, drop=True)
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    vessel_name = data["vessel_name"].values[0]
    voyage_name = i.split('processed')[0]
    speed_std = data["speed"].std()
    
    
    
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data['timestamp_seconds'] = (data['timestamp'] - pd.Timestamp("1970-01-01")) / pd.Timedelta(seconds=1)

    # Create a function that performs the interpolation
    interp_longitude = interpolate.interp1d(data['timestamp_seconds'], data['longitude'])
    interp_latitude = interpolate.interp1d(data['timestamp_seconds'], data['latitude'])
    interp_speed = interpolate.interp1d(data['timestamp_seconds'], data['speed'])

    # Specify the range of new x values for the interpolation
    new_x = np.linspace(data['timestamp_seconds'].min(), data['timestamp_seconds'].max(), num=int(interpolation_number))

    # Use the interpolation function to estimate y values for the new x values
    interp_longitude_y = interp_longitude(new_x)
    interp_latitude_y = interp_latitude(new_x)
    interp_speed_y = interp_speed(new_x)
    # Create new DataFrame with the interpolated values
    interp_data = pd.DataFrame({'timestamp': new_x, 'longitude': interp_longitude_y, 'latitude': interp_latitude_y, 'speed': interp_speed_y})
#     i = i.split("processed")[0]
    interp_data.to_csv(upper_china_path_dir + 'interpolated/'+ f"interpolated_{i}_below_3knot_{interpolation_number}.csv", encoding="utf-8 sig", header=True, index=False)
    
    # plot the data
#     sc = ax.scatter(interp_data['longitude'], interp_data['latitude'],c = interp_data['speed'], cmap = cmap, norm = norm, s=0.1)

#     Add axis labels
#     ax.set_xlabel('Longitude')
#     ax.set_ylabel('Latitude')
#     ax.set_title(f"{vessel_name}_{voyage_name}")
#     # Add a colorbar
#     cbar = plt.colorbar(sc,label='Speed')

#         Show the plot
#     plt.show()
        
    check = data["speed"].apply(lambda x : True if x < 3 else False)
    s = data.drop(data[check].index)["speed"]
    print(f"보간법 전 일반 평균 속도 : {data['speed'].mean()}")
    print(f"보간법 전 일반 속도 표준편차: {data['speed'].std()}")
    print(f"보간법 전 저속 제거 평균 속도 : {s.mean()}")
    print(f'보간법 전 저속 제거 속도 표준편차: {s.std()}')
    print('-')
    check = interp_data["speed"].apply(lambda x : True if x < 3 else False)
    s = interp_data.drop(interp_data[check].index)["speed"]
    print(f"보간법 후 일반 평균 속도 : {interp_data['speed'].mean()}")
    print(f"보간법 후 일반 속도 표준편차: {interp_data['speed'].std()}")
    print(f"보간법 후 저속 제거 평균 속도 : {s.mean()}")
    mean_speed_dict['voyage'].append(voyage_name)
    mean_speed_dict['mean_speed'].append(s.mean())
    print(f'보간법 후 저속 제거 속도 표준편차: {s.std()}')

보간법 전 일반 평균 속도 : 9.372818311874106
보간법 전 일반 속도 표준편차: 3.188774981913939
보간법 전 저속 제거 평균 속도 : 9.372818311874106
보간법 전 저속 제거 속도 표준편차: 3.188774981913939
-
보간법 후 일반 평균 속도 : 10.041873979735712
보간법 후 일반 속도 표준편차: 2.4841795384233087
보간법 후 저속 제거 평균 속도 : 10.041873979735712
보간법 후 저속 제거 속도 표준편차: 2.4841795384233087
보간법 전 일반 평균 속도 : 8.860000000000001
보간법 전 일반 속도 표준편차: 1.4070373290383746
보간법 전 저속 제거 평균 속도 : 8.860000000000001
보간법 전 저속 제거 속도 표준편차: 1.4070373290383746
-
보간법 후 일반 평균 속도 : 8.971874833638685
보간법 후 일반 속도 표준편차: 2.2344675688714974
보간법 후 저속 제거 평균 속도 : 8.971874833638685
보간법 후 저속 제거 속도 표준편차: 2.2344675688714974


In [17]:
mean_speed_df = pd.DataFrame(mean_speed_dict)
list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage)

['AKA_BHUM_9V7282_2022_7', 'AKA_BHUM_9V7282_2022_8']

In [18]:
high_speed_path_data =[]
low_speed_path_data =[]
for i in AKA_BHUM_voyages_list:
    i = i.split("processed")[0]
    if i in list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage):
        i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
        df = pd.read_csv(upper_china_path_dir + i + ".csv")
        low_speed_path_data.append(df)
#     elif i in list(mean_speed_df[mean_speed_df["mean_speed"] >= 14].voyage):
#         i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
#         df = pd.read_csv(upper_china_path_dir + i + ".csv")
#         high_speed_path_data.append(df)

In [19]:
all_data = pd.concat(low_speed_path_data)

# Add a new index column representing the row number
all_data['index'] = range(len(all_data))
all_data["index"] = all_data.index
concaternated_df = all_data.groupby('index').mean()
concaternated_df.to_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv", encoding="utf-8 sig",header=True, index=False)

## below 3 knot 제거

In [20]:
interpolation_number = "2000"

### interpolation 2000

In [21]:
max_speed_list = []
min_speed_list = []
for i in AKA_BHUM_voyages_list:
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    max_speed_list.append(max_speed)
    min_speed_list.append(min_speed)
min_speed = np.min(min_speed_list)
max_speed = np.max(max_speed_list)

# norm = plt.Normalize(min_speed, max_speed)
# cmap = plt.get_cmap('RdYlGn')
# sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
# Create a ScalarMappable with the colormap and normalization

mean_speed_dict = {"voyage" : [],"mean_speed" : []}

for i in AKA_BHUM_voyages_list:
#     fig, ax = plt.subplots()
    
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    # 3노트 이하 데이터 삭제
    data = data[data["speed"] >= 3]
    data.reset_index(inplace=True, drop=True)
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    vessel_name = data["vessel_name"].values[0]
    voyage_name = i.split('processed')[0]
    speed_std = data["speed"].std()
    
    
    
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data['timestamp_seconds'] = (data['timestamp'] - pd.Timestamp("1970-01-01")) / pd.Timedelta(seconds=1)

    # Create a function that performs the interpolation
    interp_longitude = interpolate.interp1d(data['timestamp_seconds'], data['longitude'])
    interp_latitude = interpolate.interp1d(data['timestamp_seconds'], data['latitude'])
    interp_speed = interpolate.interp1d(data['timestamp_seconds'], data['speed'])

    # Specify the range of new x values for the interpolation
    new_x = np.linspace(data['timestamp_seconds'].min(), data['timestamp_seconds'].max(), num=int(interpolation_number))

    # Use the interpolation function to estimate y values for the new x values
    interp_longitude_y = interp_longitude(new_x)
    interp_latitude_y = interp_latitude(new_x)
    interp_speed_y = interp_speed(new_x)
    # Create new DataFrame with the interpolated values
    interp_data = pd.DataFrame({'timestamp': new_x, 'longitude': interp_longitude_y, 'latitude': interp_latitude_y, 'speed': interp_speed_y})
#     i = i.split("processed")[0]
    interp_data.to_csv(upper_china_path_dir + 'interpolated/'+ f"interpolated_{i}_below_3knot_{interpolation_number}.csv", encoding="utf-8 sig", header=True, index=False)
    
    # plot the data
#     sc = ax.scatter(interp_data['longitude'], interp_data['latitude'],c = interp_data['speed'], cmap = cmap, norm = norm, s=0.1)

#     Add axis labels
#     ax.set_xlabel('Longitude')
#     ax.set_ylabel('Latitude')
#     ax.set_title(f"{vessel_name}_{voyage_name}")
#     # Add a colorbar
#     cbar = plt.colorbar(sc,label='Speed')

#         Show the plot
#     plt.show()
        
    check = data["speed"].apply(lambda x : True if x < 3 else False)
    s = data.drop(data[check].index)["speed"]
    print(f"보간법 전 일반 평균 속도 : {data['speed'].mean()}")
    print(f"보간법 전 일반 속도 표준편차: {data['speed'].std()}")
    print(f"보간법 전 저속 제거 평균 속도 : {s.mean()}")
    print(f'보간법 전 저속 제거 속도 표준편차: {s.std()}')
    print('-')
    check = interp_data["speed"].apply(lambda x : True if x < 3 else False)
    s = interp_data.drop(interp_data[check].index)["speed"]
    print(f"보간법 후 일반 평균 속도 : {interp_data['speed'].mean()}")
    print(f"보간법 후 일반 속도 표준편차: {interp_data['speed'].std()}")
    print(f"보간법 후 저속 제거 평균 속도 : {s.mean()}")
    mean_speed_dict['voyage'].append(voyage_name)
    mean_speed_dict['mean_speed'].append(s.mean())
    print(f'보간법 후 저속 제거 속도 표준편차: {s.std()}')

보간법 전 일반 평균 속도 : 9.372818311874106
보간법 전 일반 속도 표준편차: 3.188774981913939
보간법 전 저속 제거 평균 속도 : 9.372818311874106
보간법 전 저속 제거 속도 표준편차: 3.188774981913939
-
보간법 후 일반 평균 속도 : 10.042990606852964
보간법 후 일반 속도 표준편차: 2.483951932947829
보간법 후 저속 제거 평균 속도 : 10.042990606852964
보간법 후 저속 제거 속도 표준편차: 2.483951932947829
보간법 전 일반 평균 속도 : 8.860000000000001
보간법 전 일반 속도 표준편차: 1.4070373290383746
보간법 전 저속 제거 평균 속도 : 8.860000000000001
보간법 전 저속 제거 속도 표준편차: 1.4070373290383746
-
보간법 후 일반 평균 속도 : 8.972523247415458
보간법 후 일반 속도 표준편차: 2.2340848120725494
보간법 후 저속 제거 평균 속도 : 8.972523247415458
보간법 후 저속 제거 속도 표준편차: 2.2340848120725494


In [22]:
mean_speed_df = pd.DataFrame(mean_speed_dict)
list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage)

['AKA_BHUM_9V7282_2022_7', 'AKA_BHUM_9V7282_2022_8']

In [23]:
high_speed_path_data =[]
low_speed_path_data =[]
for i in AKA_BHUM_voyages_list:
    i = i.split("processed")[0]
    if i in list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage):
        i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
        df = pd.read_csv(upper_china_path_dir + i + ".csv")
        low_speed_path_data.append(df)
#     elif i in list(mean_speed_df[mean_speed_df["mean_speed"] >= 14].voyage):
#         i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
#         df = pd.read_csv(upper_china_path_dir + i + ".csv")
#         high_speed_path_data.append(df)

In [24]:
all_data = pd.concat(low_speed_path_data)

# Add a new index column representing the row number
all_data['index'] = range(len(all_data))
all_data["index"] = all_data.index
concaternated_df = all_data.groupby('index').mean()
concaternated_df.to_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv", encoding="utf-8 sig",header=True, index=False)

## below 3 knot 제거

In [25]:
interpolation_number = "2500"

### interpolation 2500

In [26]:
max_speed_list = []
min_speed_list = []
for i in AKA_BHUM_voyages_list:
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    max_speed_list.append(max_speed)
    min_speed_list.append(min_speed)
min_speed = np.min(min_speed_list)
max_speed = np.max(max_speed_list)

# norm = plt.Normalize(min_speed, max_speed)
# cmap = plt.get_cmap('RdYlGn')
# sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
# Create a ScalarMappable with the colormap and normalization

mean_speed_dict = {"voyage" : [],"mean_speed" : []}

for i in AKA_BHUM_voyages_list:
#     fig, ax = plt.subplots()
    
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    # 3노트 이하 데이터 삭제
    data = data[data["speed"] >= 3]
    data.reset_index(inplace=True, drop=True)
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    vessel_name = data["vessel_name"].values[0]
    voyage_name = i.split('processed')[0]
    speed_std = data["speed"].std()
    
    
    
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data['timestamp_seconds'] = (data['timestamp'] - pd.Timestamp("1970-01-01")) / pd.Timedelta(seconds=1)

    # Create a function that performs the interpolation
    interp_longitude = interpolate.interp1d(data['timestamp_seconds'], data['longitude'])
    interp_latitude = interpolate.interp1d(data['timestamp_seconds'], data['latitude'])
    interp_speed = interpolate.interp1d(data['timestamp_seconds'], data['speed'])

    # Specify the range of new x values for the interpolation
    new_x = np.linspace(data['timestamp_seconds'].min(), data['timestamp_seconds'].max(), num=int(interpolation_number))

    # Use the interpolation function to estimate y values for the new x values
    interp_longitude_y = interp_longitude(new_x)
    interp_latitude_y = interp_latitude(new_x)
    interp_speed_y = interp_speed(new_x)
    # Create new DataFrame with the interpolated values
    interp_data = pd.DataFrame({'timestamp': new_x, 'longitude': interp_longitude_y, 'latitude': interp_latitude_y, 'speed': interp_speed_y})
#     i = i.split("processed")[0]
    interp_data.to_csv(upper_china_path_dir + 'interpolated/'+ f"interpolated_{i}_below_3knot_{interpolation_number}.csv", encoding="utf-8 sig", header=True, index=False)
    
    # plot the data
#     sc = ax.scatter(interp_data['longitude'], interp_data['latitude'],c = interp_data['speed'], cmap = cmap, norm = norm, s=0.1)

#     Add axis labels
#     ax.set_xlabel('Longitude')
#     ax.set_ylabel('Latitude')
#     ax.set_title(f"{vessel_name}_{voyage_name}")
#     # Add a colorbar
#     cbar = plt.colorbar(sc,label='Speed')

#         Show the plot
#     plt.show()
        
    check = data["speed"].apply(lambda x : True if x < 3 else False)
    s = data.drop(data[check].index)["speed"]
    print(f"보간법 전 일반 평균 속도 : {data['speed'].mean()}")
    print(f"보간법 전 일반 속도 표준편차: {data['speed'].std()}")
    print(f"보간법 전 저속 제거 평균 속도 : {s.mean()}")
    print(f'보간법 전 저속 제거 속도 표준편차: {s.std()}')
    print('-')
    check = interp_data["speed"].apply(lambda x : True if x < 3 else False)
    s = interp_data.drop(interp_data[check].index)["speed"]
    print(f"보간법 후 일반 평균 속도 : {interp_data['speed'].mean()}")
    print(f"보간법 후 일반 속도 표준편차: {interp_data['speed'].std()}")
    print(f"보간법 후 저속 제거 평균 속도 : {s.mean()}")
    mean_speed_dict['voyage'].append(voyage_name)
    mean_speed_dict['mean_speed'].append(s.mean())
    print(f'보간법 후 저속 제거 속도 표준편차: {s.std()}')

보간법 전 일반 평균 속도 : 9.372818311874106
보간법 전 일반 속도 표준편차: 3.188774981913939
보간법 전 저속 제거 평균 속도 : 9.372818311874106
보간법 전 저속 제거 속도 표준편차: 3.188774981913939
-
보간법 후 일반 평균 속도 : 10.043147920411773
보간법 후 일반 속도 표준편차: 2.483944395941176
보간법 후 저속 제거 평균 속도 : 10.043147920411773
보간법 후 저속 제거 속도 표준편차: 2.483944395941176
보간법 전 일반 평균 속도 : 8.860000000000001
보간법 전 일반 속도 표준편차: 1.4070373290383746
보간법 전 저속 제거 평균 속도 : 8.860000000000001
보간법 전 저속 제거 속도 표준편차: 1.4070373290383746
-
보간법 후 일반 평균 속도 : 8.972580717443972
보간법 후 일반 속도 표준편차: 2.2340423196102295
보간법 후 저속 제거 평균 속도 : 8.972580717443972
보간법 후 저속 제거 속도 표준편차: 2.2340423196102295


In [27]:
mean_speed_df = pd.DataFrame(mean_speed_dict)
list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage)

['AKA_BHUM_9V7282_2022_7', 'AKA_BHUM_9V7282_2022_8']

In [28]:
high_speed_path_data =[]
low_speed_path_data =[]
for i in AKA_BHUM_voyages_list:
    i = i.split("processed")[0]
    if i in list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage):
        i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
        df = pd.read_csv(upper_china_path_dir + i + ".csv")
        low_speed_path_data.append(df)
#     elif i in list(mean_speed_df[mean_speed_df["mean_speed"] >= 14].voyage):
#         i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
#         df = pd.read_csv(upper_china_path_dir + i + ".csv")
#         high_speed_path_data.append(df)

In [29]:
all_data = pd.concat(low_speed_path_data)

# Add a new index column representing the row number
all_data['index'] = range(len(all_data))
all_data["index"] = all_data.index
concaternated_df = all_data.groupby('index').mean()
concaternated_df.to_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv", encoding="utf-8 sig",header=True, index=False)

## below 3 knot 제거

In [30]:
interpolation_number = "3000"

### interpolation 3000

In [31]:
max_speed_list = []
min_speed_list = []
for i in AKA_BHUM_voyages_list:
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    max_speed_list.append(max_speed)
    min_speed_list.append(min_speed)
min_speed = np.min(min_speed_list)
max_speed = np.max(max_speed_list)

# norm = plt.Normalize(min_speed, max_speed)
# cmap = plt.get_cmap('RdYlGn')
# sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
# Create a ScalarMappable with the colormap and normalization

mean_speed_dict = {"voyage" : [],"mean_speed" : []}

for i in AKA_BHUM_voyages_list:
#     fig, ax = plt.subplots()
    
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    # 3노트 이하 데이터 삭제
    data = data[data["speed"] >= 3]
    data.reset_index(inplace=True, drop=True)
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    vessel_name = data["vessel_name"].values[0]
    voyage_name = i.split('processed')[0]
    speed_std = data["speed"].std()
    
    
    
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data['timestamp_seconds'] = (data['timestamp'] - pd.Timestamp("1970-01-01")) / pd.Timedelta(seconds=1)

    # Create a function that performs the interpolation
    interp_longitude = interpolate.interp1d(data['timestamp_seconds'], data['longitude'])
    interp_latitude = interpolate.interp1d(data['timestamp_seconds'], data['latitude'])
    interp_speed = interpolate.interp1d(data['timestamp_seconds'], data['speed'])

    # Specify the range of new x values for the interpolation
    new_x = np.linspace(data['timestamp_seconds'].min(), data['timestamp_seconds'].max(), num=int(interpolation_number))

    # Use the interpolation function to estimate y values for the new x values
    interp_longitude_y = interp_longitude(new_x)
    interp_latitude_y = interp_latitude(new_x)
    interp_speed_y = interp_speed(new_x)
    # Create new DataFrame with the interpolated values
    interp_data = pd.DataFrame({'timestamp': new_x, 'longitude': interp_longitude_y, 'latitude': interp_latitude_y, 'speed': interp_speed_y})
#     i = i.split("processed")[0]
    interp_data.to_csv(upper_china_path_dir + 'interpolated/'+ f"interpolated_{i}_below_3knot_{interpolation_number}.csv", encoding="utf-8 sig", header=True, index=False)
    
    # plot the data
#     sc = ax.scatter(interp_data['longitude'], interp_data['latitude'],c = interp_data['speed'], cmap = cmap, norm = norm, s=0.1)

#     Add axis labels
#     ax.set_xlabel('Longitude')
#     ax.set_ylabel('Latitude')
#     ax.set_title(f"{vessel_name}_{voyage_name}")
#     # Add a colorbar
#     cbar = plt.colorbar(sc,label='Speed')

#         Show the plot
#     plt.show()
        
    check = data["speed"].apply(lambda x : True if x < 3 else False)
    s = data.drop(data[check].index)["speed"]
    print(f"보간법 전 일반 평균 속도 : {data['speed'].mean()}")
    print(f"보간법 전 일반 속도 표준편차: {data['speed'].std()}")
    print(f"보간법 전 저속 제거 평균 속도 : {s.mean()}")
    print(f'보간법 전 저속 제거 속도 표준편차: {s.std()}')
    print('-')
    check = interp_data["speed"].apply(lambda x : True if x < 3 else False)
    s = interp_data.drop(interp_data[check].index)["speed"]
    print(f"보간법 후 일반 평균 속도 : {interp_data['speed'].mean()}")
    print(f"보간법 후 일반 속도 표준편차: {interp_data['speed'].std()}")
    print(f"보간법 후 저속 제거 평균 속도 : {s.mean()}")
    mean_speed_dict['voyage'].append(voyage_name)
    mean_speed_dict['mean_speed'].append(s.mean())
    print(f'보간법 후 저속 제거 속도 표준편차: {s.std()}')

보간법 전 일반 평균 속도 : 9.372818311874106
보간법 전 일반 속도 표준편차: 3.188774981913939
보간법 전 저속 제거 평균 속도 : 9.372818311874106
보간법 전 저속 제거 속도 표준편차: 3.188774981913939
-
보간법 후 일반 평균 속도 : 10.043349718516307
보간법 후 일반 속도 표준편차: 2.484112083877818
보간법 후 저속 제거 평균 속도 : 10.043349718516307
보간법 후 저속 제거 속도 표준편차: 2.484112083877818
보간법 전 일반 평균 속도 : 8.860000000000001
보간법 전 일반 속도 표준편차: 1.4070373290383746
보간법 전 저속 제거 평균 속도 : 8.860000000000001
보간법 전 저속 제거 속도 표준편차: 1.4070373290383746
-
보간법 후 일반 평균 속도 : 8.972777809180707
보간법 후 일반 속도 표준편차: 2.2338445914937615
보간법 후 저속 제거 평균 속도 : 8.972777809180707
보간법 후 저속 제거 속도 표준편차: 2.2338445914937615


In [32]:
mean_speed_df = pd.DataFrame(mean_speed_dict)
list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage)

['AKA_BHUM_9V7282_2022_7', 'AKA_BHUM_9V7282_2022_8']

In [33]:
high_speed_path_data =[]
low_speed_path_data =[]
for i in AKA_BHUM_voyages_list:
    i = i.split("processed")[0]
    if i in list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage):
        i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
        df = pd.read_csv(upper_china_path_dir + i + ".csv")
        low_speed_path_data.append(df)
#     elif i in list(mean_speed_df[mean_speed_df["mean_speed"] >= 14].voyage):
#         i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
#         df = pd.read_csv(upper_china_path_dir + i + ".csv")
#         high_speed_path_data.append(df)

In [34]:
all_data = pd.concat(low_speed_path_data)

# Add a new index column representing the row number
all_data['index'] = range(len(all_data))
all_data["index"] = all_data.index
concaternated_df = all_data.groupby('index').mean()
concaternated_df.to_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv", encoding="utf-8 sig",header=True, index=False)

## below 3 knot 제거

In [35]:
interpolation_number = "4000"

### interpolation 4000

In [36]:
max_speed_list = []
min_speed_list = []
for i in AKA_BHUM_voyages_list:
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    max_speed_list.append(max_speed)
    min_speed_list.append(min_speed)
min_speed = np.min(min_speed_list)
max_speed = np.max(max_speed_list)

# norm = plt.Normalize(min_speed, max_speed)
# cmap = plt.get_cmap('RdYlGn')
# sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
# Create a ScalarMappable with the colormap and normalization

mean_speed_dict = {"voyage" : [],"mean_speed" : []}

for i in AKA_BHUM_voyages_list:
#     fig, ax = plt.subplots()
    
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    # 3노트 이하 데이터 삭제
    data = data[data["speed"] >= 3]
    data.reset_index(inplace=True, drop=True)
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    vessel_name = data["vessel_name"].values[0]
    voyage_name = i.split('processed')[0]
    speed_std = data["speed"].std()
    
    
    
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data['timestamp_seconds'] = (data['timestamp'] - pd.Timestamp("1970-01-01")) / pd.Timedelta(seconds=1)

    # Create a function that performs the interpolation
    interp_longitude = interpolate.interp1d(data['timestamp_seconds'], data['longitude'])
    interp_latitude = interpolate.interp1d(data['timestamp_seconds'], data['latitude'])
    interp_speed = interpolate.interp1d(data['timestamp_seconds'], data['speed'])

    # Specify the range of new x values for the interpolation
    new_x = np.linspace(data['timestamp_seconds'].min(), data['timestamp_seconds'].max(), num=int(interpolation_number))

    # Use the interpolation function to estimate y values for the new x values
    interp_longitude_y = interp_longitude(new_x)
    interp_latitude_y = interp_latitude(new_x)
    interp_speed_y = interp_speed(new_x)
    # Create new DataFrame with the interpolated values
    interp_data = pd.DataFrame({'timestamp': new_x, 'longitude': interp_longitude_y, 'latitude': interp_latitude_y, 'speed': interp_speed_y})
#     i = i.split("processed")[0]
    interp_data.to_csv(upper_china_path_dir + 'interpolated/'+ f"interpolated_{i}_below_3knot_{interpolation_number}.csv", encoding="utf-8 sig", header=True, index=False)
    
    # plot the data
#     sc = ax.scatter(interp_data['longitude'], interp_data['latitude'],c = interp_data['speed'], cmap = cmap, norm = norm, s=0.1)

#     Add axis labels
#     ax.set_xlabel('Longitude')
#     ax.set_ylabel('Latitude')
#     ax.set_title(f"{vessel_name}_{voyage_name}")
#     # Add a colorbar
#     cbar = plt.colorbar(sc,label='Speed')

#         Show the plot
#     plt.show()
        
    check = data["speed"].apply(lambda x : True if x < 3 else False)
    s = data.drop(data[check].index)["speed"]
    print(f"보간법 전 일반 평균 속도 : {data['speed'].mean()}")
    print(f"보간법 전 일반 속도 표준편차: {data['speed'].std()}")
    print(f"보간법 전 저속 제거 평균 속도 : {s.mean()}")
    print(f'보간법 전 저속 제거 속도 표준편차: {s.std()}')
    print('-')
    check = interp_data["speed"].apply(lambda x : True if x < 3 else False)
    s = interp_data.drop(interp_data[check].index)["speed"]
    print(f"보간법 후 일반 평균 속도 : {interp_data['speed'].mean()}")
    print(f"보간법 후 일반 속도 표준편차: {interp_data['speed'].std()}")
    print(f"보간법 후 저속 제거 평균 속도 : {s.mean()}")
    mean_speed_dict['voyage'].append(voyage_name)
    mean_speed_dict['mean_speed'].append(s.mean())
    print(f'보간법 후 저속 제거 속도 표준편차: {s.std()}')

보간법 전 일반 평균 속도 : 9.372818311874106
보간법 전 일반 속도 표준편차: 3.188774981913939
보간법 전 저속 제거 평균 속도 : 9.372818311874106
보간법 전 저속 제거 속도 표준편차: 3.188774981913939
-
보간법 후 일반 평균 속도 : 10.043471865457862
보간법 후 일반 속도 표준편차: 2.483884212529587
보간법 후 저속 제거 평균 속도 : 10.043471865457862
보간법 후 저속 제거 속도 표준편차: 2.483884212529587
보간법 전 일반 평균 속도 : 8.860000000000001
보간법 전 일반 속도 표준편차: 1.4070373290383746
보간법 전 저속 제거 평균 속도 : 8.860000000000001
보간법 전 저속 제거 속도 표준편차: 1.4070373290383746
-
보간법 후 일반 평균 속도 : 8.97285097784577
보간법 후 일반 속도 표준편차: 2.2337798705430343
보간법 후 저속 제거 평균 속도 : 8.97285097784577
보간법 후 저속 제거 속도 표준편차: 2.2337798705430343


In [37]:
mean_speed_df = pd.DataFrame(mean_speed_dict)
list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage)

['AKA_BHUM_9V7282_2022_7', 'AKA_BHUM_9V7282_2022_8']

In [38]:
high_speed_path_data =[]
low_speed_path_data =[]
for i in AKA_BHUM_voyages_list:
    i = i.split("processed")[0]
    if i in list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage):
        i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
        df = pd.read_csv(upper_china_path_dir + i + ".csv")
        low_speed_path_data.append(df)
#     elif i in list(mean_speed_df[mean_speed_df["mean_speed"] >= 14].voyage):
#         i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
#         df = pd.read_csv(upper_china_path_dir + i + ".csv")
#         high_speed_path_data.append(df)

In [39]:
all_data = pd.concat(low_speed_path_data)

# Add a new index column representing the row number
all_data['index'] = range(len(all_data))
all_data["index"] = all_data.index
concaternated_df = all_data.groupby('index').mean()
concaternated_df.to_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv", encoding="utf-8 sig",header=True, index=False)

## below 3 knot 제거

In [40]:
interpolation_number = "5000"

### interpolation 5000

In [41]:
max_speed_list = []
min_speed_list = []
for i in AKA_BHUM_voyages_list:
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    max_speed_list.append(max_speed)
    min_speed_list.append(min_speed)
min_speed = np.min(min_speed_list)
max_speed = np.max(max_speed_list)

# norm = plt.Normalize(min_speed, max_speed)
# cmap = plt.get_cmap('RdYlGn')
# sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
# Create a ScalarMappable with the colormap and normalization

mean_speed_dict = {"voyage" : [],"mean_speed" : []}

for i in AKA_BHUM_voyages_list:
#     fig, ax = plt.subplots()
    
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    # 3노트 이하 데이터 삭제
    data = data[data["speed"] >= 3]
    data.reset_index(inplace=True, drop=True)
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    vessel_name = data["vessel_name"].values[0]
    voyage_name = i.split('processed')[0]
    speed_std = data["speed"].std()
    
    
    
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data['timestamp_seconds'] = (data['timestamp'] - pd.Timestamp("1970-01-01")) / pd.Timedelta(seconds=1)

    # Create a function that performs the interpolation
    interp_longitude = interpolate.interp1d(data['timestamp_seconds'], data['longitude'])
    interp_latitude = interpolate.interp1d(data['timestamp_seconds'], data['latitude'])
    interp_speed = interpolate.interp1d(data['timestamp_seconds'], data['speed'])

    # Specify the range of new x values for the interpolation
    new_x = np.linspace(data['timestamp_seconds'].min(), data['timestamp_seconds'].max(), num=int(interpolation_number))

    # Use the interpolation function to estimate y values for the new x values
    interp_longitude_y = interp_longitude(new_x)
    interp_latitude_y = interp_latitude(new_x)
    interp_speed_y = interp_speed(new_x)
    # Create new DataFrame with the interpolated values
    interp_data = pd.DataFrame({'timestamp': new_x, 'longitude': interp_longitude_y, 'latitude': interp_latitude_y, 'speed': interp_speed_y})
#     i = i.split("processed")[0]
    interp_data.to_csv(upper_china_path_dir + 'interpolated/'+ f"interpolated_{i}_below_3knot_{interpolation_number}.csv", encoding="utf-8 sig", header=True, index=False)
    
    # plot the data
#     sc = ax.scatter(interp_data['longitude'], interp_data['latitude'],c = interp_data['speed'], cmap = cmap, norm = norm, s=0.1)

#     Add axis labels
#     ax.set_xlabel('Longitude')
#     ax.set_ylabel('Latitude')
#     ax.set_title(f"{vessel_name}_{voyage_name}")
#     # Add a colorbar
#     cbar = plt.colorbar(sc,label='Speed')

#         Show the plot
#     plt.show()
        
    check = data["speed"].apply(lambda x : True if x < 3 else False)
    s = data.drop(data[check].index)["speed"]
    print(f"보간법 전 일반 평균 속도 : {data['speed'].mean()}")
    print(f"보간법 전 일반 속도 표준편차: {data['speed'].std()}")
    print(f"보간법 전 저속 제거 평균 속도 : {s.mean()}")
    print(f'보간법 전 저속 제거 속도 표준편차: {s.std()}')
    print('-')
    check = interp_data["speed"].apply(lambda x : True if x < 3 else False)
    s = interp_data.drop(interp_data[check].index)["speed"]
    print(f"보간법 후 일반 평균 속도 : {interp_data['speed'].mean()}")
    print(f"보간법 후 일반 속도 표준편차: {interp_data['speed'].std()}")
    print(f"보간법 후 저속 제거 평균 속도 : {s.mean()}")
    mean_speed_dict['voyage'].append(voyage_name)
    mean_speed_dict['mean_speed'].append(s.mean())
    print(f'보간법 후 저속 제거 속도 표준편차: {s.std()}')

보간법 전 일반 평균 속도 : 9.372818311874106
보간법 전 일반 속도 표준편차: 3.188774981913939
보간법 전 저속 제거 평균 속도 : 9.372818311874106
보간법 전 저속 제거 속도 표준편차: 3.188774981913939
-
보간법 후 일반 평균 속도 : 10.043564349640501
보간법 후 일반 속도 표준편차: 2.4838395141135976
보간법 후 저속 제거 평균 속도 : 10.043564349640501
보간법 후 저속 제거 속도 표준편차: 2.4838395141135976
보간법 전 일반 평균 속도 : 8.860000000000001
보간법 전 일반 속도 표준편차: 1.4070373290383746
보간법 전 저속 제거 평균 속도 : 8.860000000000001
보간법 전 저속 제거 속도 표준편차: 1.4070373290383746
-
보간법 후 일반 평균 속도 : 8.97290890677493
보간법 후 일반 속도 표준편차: 2.2336914344566554
보간법 후 저속 제거 평균 속도 : 8.97290890677493
보간법 후 저속 제거 속도 표준편차: 2.2336914344566554


In [42]:
mean_speed_df = pd.DataFrame(mean_speed_dict)
list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage)

['AKA_BHUM_9V7282_2022_7', 'AKA_BHUM_9V7282_2022_8']

In [43]:
high_speed_path_data =[]
low_speed_path_data =[]
for i in AKA_BHUM_voyages_list:
    i = i.split("processed")[0]
    if i in list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage):
        i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
        df = pd.read_csv(upper_china_path_dir + i + ".csv")
        low_speed_path_data.append(df)
#     elif i in list(mean_speed_df[mean_speed_df["mean_speed"] >= 14].voyage):
#         i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
#         df = pd.read_csv(upper_china_path_dir + i + ".csv")
#         high_speed_path_data.append(df)

In [44]:
all_data = pd.concat(low_speed_path_data)

# Add a new index column representing the row number
all_data['index'] = range(len(all_data))
all_data["index"] = all_data.index
concaternated_df = all_data.groupby('index').mean()
concaternated_df.to_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv", encoding="utf-8 sig",header=True, index=False)

## below 3 knot 제거

In [45]:
interpolation_number = "7500"

### interpolation 7500

In [46]:
max_speed_list = []
min_speed_list = []
for i in AKA_BHUM_voyages_list:
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    max_speed_list.append(max_speed)
    min_speed_list.append(min_speed)
min_speed = np.min(min_speed_list)
max_speed = np.max(max_speed_list)

# norm = plt.Normalize(min_speed, max_speed)
# cmap = plt.get_cmap('RdYlGn')
# sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
# Create a ScalarMappable with the colormap and normalization

mean_speed_dict = {"voyage" : [],"mean_speed" : []}

for i in AKA_BHUM_voyages_list:
#     fig, ax = plt.subplots()
    
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    # 3노트 이하 데이터 삭제
    data = data[data["speed"] >= 3]
    data.reset_index(inplace=True, drop=True)
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    vessel_name = data["vessel_name"].values[0]
    voyage_name = i.split('processed')[0]
    speed_std = data["speed"].std()
    
    
    
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data['timestamp_seconds'] = (data['timestamp'] - pd.Timestamp("1970-01-01")) / pd.Timedelta(seconds=1)

    # Create a function that performs the interpolation
    interp_longitude = interpolate.interp1d(data['timestamp_seconds'], data['longitude'])
    interp_latitude = interpolate.interp1d(data['timestamp_seconds'], data['latitude'])
    interp_speed = interpolate.interp1d(data['timestamp_seconds'], data['speed'])

    # Specify the range of new x values for the interpolation
    new_x = np.linspace(data['timestamp_seconds'].min(), data['timestamp_seconds'].max(), num=int(interpolation_number))

    # Use the interpolation function to estimate y values for the new x values
    interp_longitude_y = interp_longitude(new_x)
    interp_latitude_y = interp_latitude(new_x)
    interp_speed_y = interp_speed(new_x)
    # Create new DataFrame with the interpolated values
    interp_data = pd.DataFrame({'timestamp': new_x, 'longitude': interp_longitude_y, 'latitude': interp_latitude_y, 'speed': interp_speed_y})
#     i = i.split("processed")[0]
    interp_data.to_csv(upper_china_path_dir + 'interpolated/'+ f"interpolated_{i}_below_3knot_{interpolation_number}.csv", encoding="utf-8 sig", header=True, index=False)
    
    # plot the data
#     sc = ax.scatter(interp_data['longitude'], interp_data['latitude'],c = interp_data['speed'], cmap = cmap, norm = norm, s=0.1)

#     Add axis labels
#     ax.set_xlabel('Longitude')
#     ax.set_ylabel('Latitude')
#     ax.set_title(f"{vessel_name}_{voyage_name}")
#     # Add a colorbar
#     cbar = plt.colorbar(sc,label='Speed')

#         Show the plot
#     plt.show()
        
    check = data["speed"].apply(lambda x : True if x < 3 else False)
    s = data.drop(data[check].index)["speed"]
    print(f"보간법 전 일반 평균 속도 : {data['speed'].mean()}")
    print(f"보간법 전 일반 속도 표준편차: {data['speed'].std()}")
    print(f"보간법 전 저속 제거 평균 속도 : {s.mean()}")
    print(f'보간법 전 저속 제거 속도 표준편차: {s.std()}')
    print('-')
    check = interp_data["speed"].apply(lambda x : True if x < 3 else False)
    s = interp_data.drop(interp_data[check].index)["speed"]
    print(f"보간법 후 일반 평균 속도 : {interp_data['speed'].mean()}")
    print(f"보간법 후 일반 속도 표준편차: {interp_data['speed'].std()}")
    print(f"보간법 후 저속 제거 평균 속도 : {s.mean()}")
    mean_speed_dict['voyage'].append(voyage_name)
    mean_speed_dict['mean_speed'].append(s.mean())
    print(f'보간법 후 저속 제거 속도 표준편차: {s.std()}')

보간법 전 일반 평균 속도 : 9.372818311874106
보간법 전 일반 속도 표준편차: 3.188774981913939
보간법 전 저속 제거 평균 속도 : 9.372818311874106
보간법 전 저속 제거 속도 표준편차: 3.188774981913939
-
보간법 후 일반 평균 속도 : 10.043692952681106
보간법 후 일반 속도 표준편차: 2.4837611249767027
보간법 후 저속 제거 평균 속도 : 10.043692952681106
보간법 후 저속 제거 속도 표준편차: 2.4837611249767027
보간법 전 일반 평균 속도 : 8.860000000000001
보간법 전 일반 속도 표준편차: 1.4070373290383746
보간법 전 저속 제거 평균 속도 : 8.860000000000001
보간법 전 저속 제거 속도 표준편차: 1.4070373290383746
-
보간법 후 일반 평균 속도 : 8.972984452721867
보간법 후 일반 속도 표준편차: 2.233628509890076
보간법 후 저속 제거 평균 속도 : 8.972984452721867
보간법 후 저속 제거 속도 표준편차: 2.233628509890076


In [47]:
mean_speed_df = pd.DataFrame(mean_speed_dict)
list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage)

['AKA_BHUM_9V7282_2022_7', 'AKA_BHUM_9V7282_2022_8']

In [48]:
high_speed_path_data =[]
low_speed_path_data =[]
for i in AKA_BHUM_voyages_list:
    i = i.split("processed")[0]
    if i in list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage):
        i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
        df = pd.read_csv(upper_china_path_dir + i + ".csv")
        low_speed_path_data.append(df)
#     elif i in list(mean_speed_df[mean_speed_df["mean_speed"] >= 14].voyage):
#         i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
#         df = pd.read_csv(upper_china_path_dir + i + ".csv")
#         high_speed_path_data.append(df)

In [49]:
all_data = pd.concat(low_speed_path_data)

# Add a new index column representing the row number
all_data['index'] = range(len(all_data))
all_data["index"] = all_data.index
concaternated_df = all_data.groupby('index').mean()
concaternated_df.to_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv", encoding="utf-8 sig",header=True, index=False)

## below 3 knot 제거

In [50]:
interpolation_number = "9000"

### interpolation 9000

In [51]:
max_speed_list = []
min_speed_list = []
for i in AKA_BHUM_voyages_list:
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    max_speed_list.append(max_speed)
    min_speed_list.append(min_speed)
min_speed = np.min(min_speed_list)
max_speed = np.max(max_speed_list)

# norm = plt.Normalize(min_speed, max_speed)
# cmap = plt.get_cmap('RdYlGn')
# sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
# Create a ScalarMappable with the colormap and normalization

mean_speed_dict = {"voyage" : [],"mean_speed" : []}

for i in AKA_BHUM_voyages_list:
#     fig, ax = plt.subplots()
    
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    # 3노트 이하 데이터 삭제
    data = data[data["speed"] >= 3]
    data.reset_index(inplace=True, drop=True)
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    vessel_name = data["vessel_name"].values[0]
    voyage_name = i.split('processed')[0]
    speed_std = data["speed"].std()
    
    
    
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data['timestamp_seconds'] = (data['timestamp'] - pd.Timestamp("1970-01-01")) / pd.Timedelta(seconds=1)

    # Create a function that performs the interpolation
    interp_longitude = interpolate.interp1d(data['timestamp_seconds'], data['longitude'])
    interp_latitude = interpolate.interp1d(data['timestamp_seconds'], data['latitude'])
    interp_speed = interpolate.interp1d(data['timestamp_seconds'], data['speed'])

    # Specify the range of new x values for the interpolation
    new_x = np.linspace(data['timestamp_seconds'].min(), data['timestamp_seconds'].max(), num=int(interpolation_number))

    # Use the interpolation function to estimate y values for the new x values
    interp_longitude_y = interp_longitude(new_x)
    interp_latitude_y = interp_latitude(new_x)
    interp_speed_y = interp_speed(new_x)
    # Create new DataFrame with the interpolated values
    interp_data = pd.DataFrame({'timestamp': new_x, 'longitude': interp_longitude_y, 'latitude': interp_latitude_y, 'speed': interp_speed_y})
#     i = i.split("processed")[0]
    interp_data.to_csv(upper_china_path_dir + 'interpolated/'+ f"interpolated_{i}_below_3knot_{interpolation_number}.csv", encoding="utf-8 sig", header=True, index=False)
    
    # plot the data
#     sc = ax.scatter(interp_data['longitude'], interp_data['latitude'],c = interp_data['speed'], cmap = cmap, norm = norm, s=0.1)

#     Add axis labels
#     ax.set_xlabel('Longitude')
#     ax.set_ylabel('Latitude')
#     ax.set_title(f"{vessel_name}_{voyage_name}")
#     # Add a colorbar
#     cbar = plt.colorbar(sc,label='Speed')

#         Show the plot
#     plt.show()
        
    check = data["speed"].apply(lambda x : True if x < 3 else False)
    s = data.drop(data[check].index)["speed"]
    print(f"보간법 전 일반 평균 속도 : {data['speed'].mean()}")
    print(f"보간법 전 일반 속도 표준편차: {data['speed'].std()}")
    print(f"보간법 전 저속 제거 평균 속도 : {s.mean()}")
    print(f'보간법 전 저속 제거 속도 표준편차: {s.std()}')
    print('-')
    check = interp_data["speed"].apply(lambda x : True if x < 3 else False)
    s = interp_data.drop(interp_data[check].index)["speed"]
    print(f"보간법 후 일반 평균 속도 : {interp_data['speed'].mean()}")
    print(f"보간법 후 일반 속도 표준편차: {interp_data['speed'].std()}")
    print(f"보간법 후 저속 제거 평균 속도 : {s.mean()}")
    mean_speed_dict['voyage'].append(voyage_name)
    mean_speed_dict['mean_speed'].append(s.mean())
    print(f'보간법 후 저속 제거 속도 표준편차: {s.std()}')

보간법 전 일반 평균 속도 : 9.372818311874106
보간법 전 일반 속도 표준편차: 3.188774981913939
보간법 전 저속 제거 평균 속도 : 9.372818311874106
보간법 전 저속 제거 속도 표준편차: 3.188774981913939
-
보간법 후 일반 평균 속도 : 10.043740679537667
보간법 후 일반 속도 표준편차: 2.48373752580512
보간법 후 저속 제거 평균 속도 : 10.043740679537667
보간법 후 저속 제거 속도 표준편차: 2.48373752580512
보간법 전 일반 평균 속도 : 8.860000000000001
보간법 전 일반 속도 표준편차: 1.4070373290383746
보간법 전 저속 제거 평균 속도 : 8.860000000000001
보간법 전 저속 제거 속도 표준편차: 1.4070373290383746
-
보간법 후 일반 평균 속도 : 8.973013852879493
보간법 후 일반 속도 표준편차: 2.2336057044578754
보간법 후 저속 제거 평균 속도 : 8.973013852879493
보간법 후 저속 제거 속도 표준편차: 2.2336057044578754


In [52]:
mean_speed_df = pd.DataFrame(mean_speed_dict)
list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage)

['AKA_BHUM_9V7282_2022_7', 'AKA_BHUM_9V7282_2022_8']

In [53]:
high_speed_path_data =[]
low_speed_path_data =[]
for i in AKA_BHUM_voyages_list:
    i = i.split("processed")[0]
    if i in list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage):
        i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
        df = pd.read_csv(upper_china_path_dir + i + ".csv")
        low_speed_path_data.append(df)
#     elif i in list(mean_speed_df[mean_speed_df["mean_speed"] >= 14].voyage):
#         i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
#         df = pd.read_csv(upper_china_path_dir + i + ".csv")
#         high_speed_path_data.append(df)

In [54]:
all_data = pd.concat(low_speed_path_data)

# Add a new index column representing the row number
all_data['index'] = range(len(all_data))
all_data["index"] = all_data.index
concaternated_df = all_data.groupby('index').mean()
concaternated_df.to_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv", encoding="utf-8 sig",header=True, index=False)

## below 3 knot 제거

In [55]:
interpolation_number = "10000"

### interpolation 10000

In [56]:
max_speed_list = []
min_speed_list = []
for i in AKA_BHUM_voyages_list:
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    max_speed_list.append(max_speed)
    min_speed_list.append(min_speed)
min_speed = np.min(min_speed_list)
max_speed = np.max(max_speed_list)

# norm = plt.Normalize(min_speed, max_speed)
# cmap = plt.get_cmap('RdYlGn')
# sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
# Create a ScalarMappable with the colormap and normalization

mean_speed_dict = {"voyage" : [],"mean_speed" : []}

for i in AKA_BHUM_voyages_list:
#     fig, ax = plt.subplots()
    
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    # 3노트 이하 데이터 삭제
    data = data[data["speed"] >= 3]
    data.reset_index(inplace=True, drop=True)
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    vessel_name = data["vessel_name"].values[0]
    voyage_name = i.split('processed')[0]
    speed_std = data["speed"].std()
    
    
    
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data['timestamp_seconds'] = (data['timestamp'] - pd.Timestamp("1970-01-01")) / pd.Timedelta(seconds=1)

    # Create a function that performs the interpolation
    interp_longitude = interpolate.interp1d(data['timestamp_seconds'], data['longitude'])
    interp_latitude = interpolate.interp1d(data['timestamp_seconds'], data['latitude'])
    interp_speed = interpolate.interp1d(data['timestamp_seconds'], data['speed'])

    # Specify the range of new x values for the interpolation
    new_x = np.linspace(data['timestamp_seconds'].min(), data['timestamp_seconds'].max(), num=int(interpolation_number))

    # Use the interpolation function to estimate y values for the new x values
    interp_longitude_y = interp_longitude(new_x)
    interp_latitude_y = interp_latitude(new_x)
    interp_speed_y = interp_speed(new_x)
    # Create new DataFrame with the interpolated values
    interp_data = pd.DataFrame({'timestamp': new_x, 'longitude': interp_longitude_y, 'latitude': interp_latitude_y, 'speed': interp_speed_y})
#     i = i.split("processed")[0]
    interp_data.to_csv(upper_china_path_dir + 'interpolated/'+ f"interpolated_{i}_below_3knot_{interpolation_number}.csv", encoding="utf-8 sig", header=True, index=False)
    
    # plot the data
#     sc = ax.scatter(interp_data['longitude'], interp_data['latitude'],c = interp_data['speed'], cmap = cmap, norm = norm, s=0.1)

#     Add axis labels
#     ax.set_xlabel('Longitude')
#     ax.set_ylabel('Latitude')
#     ax.set_title(f"{vessel_name}_{voyage_name}")
#     # Add a colorbar
#     cbar = plt.colorbar(sc,label='Speed')

#         Show the plot
#     plt.show()
        
    check = data["speed"].apply(lambda x : True if x < 3 else False)
    s = data.drop(data[check].index)["speed"]
    print(f"보간법 전 일반 평균 속도 : {data['speed'].mean()}")
    print(f"보간법 전 일반 속도 표준편차: {data['speed'].std()}")
    print(f"보간법 전 저속 제거 평균 속도 : {s.mean()}")
    print(f'보간법 전 저속 제거 속도 표준편차: {s.std()}')
    print('-')
    check = interp_data["speed"].apply(lambda x : True if x < 3 else False)
    s = interp_data.drop(interp_data[check].index)["speed"]
    print(f"보간법 후 일반 평균 속도 : {interp_data['speed'].mean()}")
    print(f"보간법 후 일반 속도 표준편차: {interp_data['speed'].std()}")
    print(f"보간법 후 저속 제거 평균 속도 : {s.mean()}")
    mean_speed_dict['voyage'].append(voyage_name)
    mean_speed_dict['mean_speed'].append(s.mean())
    print(f'보간법 후 저속 제거 속도 표준편차: {s.std()}')

보간법 전 일반 평균 속도 : 9.372818311874106
보간법 전 일반 속도 표준편차: 3.188774981913939
보간법 전 저속 제거 평균 속도 : 9.372818311874106
보간법 전 저속 제거 속도 표준편차: 3.188774981913939
-
보간법 후 일반 평균 속도 : 10.043762945694306
보간법 후 일반 속도 표준편차: 2.483736978983716
보간법 후 저속 제거 평균 속도 : 10.043762945694306
보간법 후 저속 제거 속도 표준편차: 2.483736978983716
보간법 전 일반 평균 속도 : 8.860000000000001
보간법 전 일반 속도 표준편차: 1.4070373290383746
보간법 전 저속 제거 평균 속도 : 8.860000000000001
보간법 전 저속 제거 속도 표준편차: 1.4070373290383746
-
보간법 후 일반 평균 속도 : 8.973029394002777
보간법 후 일반 속도 표준편차: 2.2335932796100963
보간법 후 저속 제거 평균 속도 : 8.973029394002777
보간법 후 저속 제거 속도 표준편차: 2.2335932796100963


In [57]:
mean_speed_df = pd.DataFrame(mean_speed_dict)
list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage)

['AKA_BHUM_9V7282_2022_7', 'AKA_BHUM_9V7282_2022_8']

In [58]:
high_speed_path_data =[]
low_speed_path_data =[]
for i in AKA_BHUM_voyages_list:
    i = i.split("processed")[0]
    if i in list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage):
        i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
        df = pd.read_csv(upper_china_path_dir + i + ".csv")
        low_speed_path_data.append(df)
#     elif i in list(mean_speed_df[mean_speed_df["mean_speed"] >= 14].voyage):
#         i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
#         df = pd.read_csv(upper_china_path_dir + i + ".csv")
#         high_speed_path_data.append(df)

In [59]:
all_data = pd.concat(low_speed_path_data)

# Add a new index column representing the row number
all_data['index'] = range(len(all_data))
all_data["index"] = all_data.index
concaternated_df = all_data.groupby('index').mean()
concaternated_df.to_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv", encoding="utf-8 sig",header=True, index=False)

## below 3 knot 제거

In [60]:
interpolation_number = "11000"

### interpolation 11000

In [61]:
max_speed_list = []
min_speed_list = []
for i in AKA_BHUM_voyages_list:
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    max_speed_list.append(max_speed)
    min_speed_list.append(min_speed)
min_speed = np.min(min_speed_list)
max_speed = np.max(max_speed_list)

# norm = plt.Normalize(min_speed, max_speed)
# cmap = plt.get_cmap('RdYlGn')
# sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
# Create a ScalarMappable with the colormap and normalization

mean_speed_dict = {"voyage" : [],"mean_speed" : []}

for i in AKA_BHUM_voyages_list:
#     fig, ax = plt.subplots()
    
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    # 3노트 이하 데이터 삭제
    data = data[data["speed"] >= 3]
    data.reset_index(inplace=True, drop=True)
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    vessel_name = data["vessel_name"].values[0]
    voyage_name = i.split('processed')[0]
    speed_std = data["speed"].std()
    
    
    
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data['timestamp_seconds'] = (data['timestamp'] - pd.Timestamp("1970-01-01")) / pd.Timedelta(seconds=1)

    # Create a function that performs the interpolation
    interp_longitude = interpolate.interp1d(data['timestamp_seconds'], data['longitude'])
    interp_latitude = interpolate.interp1d(data['timestamp_seconds'], data['latitude'])
    interp_speed = interpolate.interp1d(data['timestamp_seconds'], data['speed'])

    # Specify the range of new x values for the interpolation
    new_x = np.linspace(data['timestamp_seconds'].min(), data['timestamp_seconds'].max(), num=int(interpolation_number))

    # Use the interpolation function to estimate y values for the new x values
    interp_longitude_y = interp_longitude(new_x)
    interp_latitude_y = interp_latitude(new_x)
    interp_speed_y = interp_speed(new_x)
    # Create new DataFrame with the interpolated values
    interp_data = pd.DataFrame({'timestamp': new_x, 'longitude': interp_longitude_y, 'latitude': interp_latitude_y, 'speed': interp_speed_y})
#     i = i.split("processed")[0]
    interp_data.to_csv(upper_china_path_dir + 'interpolated/'+ f"interpolated_{i}_below_3knot_{interpolation_number}.csv", encoding="utf-8 sig", header=True, index=False)
    
    # plot the data
#     sc = ax.scatter(interp_data['longitude'], interp_data['latitude'],c = interp_data['speed'], cmap = cmap, norm = norm, s=0.1)

#     Add axis labels
#     ax.set_xlabel('Longitude')
#     ax.set_ylabel('Latitude')
#     ax.set_title(f"{vessel_name}_{voyage_name}")
#     # Add a colorbar
#     cbar = plt.colorbar(sc,label='Speed')

#         Show the plot
#     plt.show()
        
    check = data["speed"].apply(lambda x : True if x < 3 else False)
    s = data.drop(data[check].index)["speed"]
    print(f"보간법 전 일반 평균 속도 : {data['speed'].mean()}")
    print(f"보간법 전 일반 속도 표준편차: {data['speed'].std()}")
    print(f"보간법 전 저속 제거 평균 속도 : {s.mean()}")
    print(f'보간법 전 저속 제거 속도 표준편차: {s.std()}')
    print('-')
    check = interp_data["speed"].apply(lambda x : True if x < 3 else False)
    s = interp_data.drop(interp_data[check].index)["speed"]
    print(f"보간법 후 일반 평균 속도 : {interp_data['speed'].mean()}")
    print(f"보간법 후 일반 속도 표준편차: {interp_data['speed'].std()}")
    print(f"보간법 후 저속 제거 평균 속도 : {s.mean()}")
    mean_speed_dict['voyage'].append(voyage_name)
    mean_speed_dict['mean_speed'].append(s.mean())
    print(f'보간법 후 저속 제거 속도 표준편차: {s.std()}')

보간법 전 일반 평균 속도 : 9.372818311874106
보간법 전 일반 속도 표준편차: 3.188774981913939
보간법 전 저속 제거 평균 속도 : 9.372818311874106
보간법 전 저속 제거 속도 표준편차: 3.188774981913939
-
보간법 후 일반 평균 속도 : 10.04378243069384
보간법 후 일반 속도 표준편차: 2.4837263557906324
보간법 후 저속 제거 평균 속도 : 10.04378243069384
보간법 후 저속 제거 속도 표준편차: 2.4837263557906324
보간법 전 일반 평균 속도 : 8.860000000000001
보간법 전 일반 속도 표준편차: 1.4070373290383746
보간법 전 저속 제거 평균 속도 : 8.860000000000001
보간법 전 저속 제거 속도 표준편차: 1.4070373290383746
-
보간법 후 일반 평균 속도 : 8.973038685696448
보간법 후 일반 속도 표준편차: 2.2335852549448147
보간법 후 저속 제거 평균 속도 : 8.973038685696448
보간법 후 저속 제거 속도 표준편차: 2.2335852549448147


In [62]:
mean_speed_df = pd.DataFrame(mean_speed_dict)
list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage)

['AKA_BHUM_9V7282_2022_7', 'AKA_BHUM_9V7282_2022_8']

In [63]:
high_speed_path_data =[]
low_speed_path_data =[]
for i in AKA_BHUM_voyages_list:
    i = i.split("processed")[0]
    if i in list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage):
        i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
        df = pd.read_csv(upper_china_path_dir + i + ".csv")
        low_speed_path_data.append(df)
#     elif i in list(mean_speed_df[mean_speed_df["mean_speed"] >= 14].voyage):
#         i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
#         df = pd.read_csv(upper_china_path_dir + i + ".csv")
#         high_speed_path_data.append(df)

In [64]:
all_data = pd.concat(low_speed_path_data)

# Add a new index column representing the row number
all_data['index'] = range(len(all_data))
all_data["index"] = all_data.index
concaternated_df = all_data.groupby('index').mean()
concaternated_df.to_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv", encoding="utf-8 sig",header=True, index=False)

## below 3 knot 제거

In [65]:
interpolation_number = "12000"

### interpolation 12000

In [66]:
max_speed_list = []
min_speed_list = []
for i in AKA_BHUM_voyages_list:
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    max_speed_list.append(max_speed)
    min_speed_list.append(min_speed)
min_speed = np.min(min_speed_list)
max_speed = np.max(max_speed_list)

# norm = plt.Normalize(min_speed, max_speed)
# cmap = plt.get_cmap('RdYlGn')
# sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
# Create a ScalarMappable with the colormap and normalization

mean_speed_dict = {"voyage" : [],"mean_speed" : []}

for i in AKA_BHUM_voyages_list:
#     fig, ax = plt.subplots()
    
    df = pd.read_csv(upper_china_path_dir + i + ".csv")
    data = df
    # 3노트 이하 데이터 삭제
    data = data[data["speed"] >= 3]
    data.reset_index(inplace=True, drop=True)
    min_speed = data["speed"].min()
    max_speed = data["speed"].max()
    vessel_name = data["vessel_name"].values[0]
    voyage_name = i.split('processed')[0]
    speed_std = data["speed"].std()
    
    
    
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data['timestamp_seconds'] = (data['timestamp'] - pd.Timestamp("1970-01-01")) / pd.Timedelta(seconds=1)

    # Create a function that performs the interpolation
    interp_longitude = interpolate.interp1d(data['timestamp_seconds'], data['longitude'])
    interp_latitude = interpolate.interp1d(data['timestamp_seconds'], data['latitude'])
    interp_speed = interpolate.interp1d(data['timestamp_seconds'], data['speed'])

    # Specify the range of new x values for the interpolation
    new_x = np.linspace(data['timestamp_seconds'].min(), data['timestamp_seconds'].max(), num=int(interpolation_number))

    # Use the interpolation function to estimate y values for the new x values
    interp_longitude_y = interp_longitude(new_x)
    interp_latitude_y = interp_latitude(new_x)
    interp_speed_y = interp_speed(new_x)
    # Create new DataFrame with the interpolated values
    interp_data = pd.DataFrame({'timestamp': new_x, 'longitude': interp_longitude_y, 'latitude': interp_latitude_y, 'speed': interp_speed_y})
#     i = i.split("processed")[0]
    interp_data.to_csv(upper_china_path_dir + 'interpolated/'+ f"interpolated_{i}_below_3knot_{interpolation_number}.csv", encoding="utf-8 sig", header=True, index=False)
    
    # plot the data
#     sc = ax.scatter(interp_data['longitude'], interp_data['latitude'],c = interp_data['speed'], cmap = cmap, norm = norm, s=0.1)

#     Add axis labels
#     ax.set_xlabel('Longitude')
#     ax.set_ylabel('Latitude')
#     ax.set_title(f"{vessel_name}_{voyage_name}")
#     # Add a colorbar
#     cbar = plt.colorbar(sc,label='Speed')

#         Show the plot
#     plt.show()
        
    check = data["speed"].apply(lambda x : True if x < 3 else False)
    s = data.drop(data[check].index)["speed"]
    print(f"보간법 전 일반 평균 속도 : {data['speed'].mean()}")
    print(f"보간법 전 일반 속도 표준편차: {data['speed'].std()}")
    print(f"보간법 전 저속 제거 평균 속도 : {s.mean()}")
    print(f'보간법 전 저속 제거 속도 표준편차: {s.std()}')
    print('-')
    check = interp_data["speed"].apply(lambda x : True if x < 3 else False)
    s = interp_data.drop(interp_data[check].index)["speed"]
    print(f"보간법 후 일반 평균 속도 : {interp_data['speed'].mean()}")
    print(f"보간법 후 일반 속도 표준편차: {interp_data['speed'].std()}")
    print(f"보간법 후 저속 제거 평균 속도 : {s.mean()}")
    mean_speed_dict['voyage'].append(voyage_name)
    mean_speed_dict['mean_speed'].append(s.mean())
    print(f'보간법 후 저속 제거 속도 표준편차: {s.std()}')

보간법 전 일반 평균 속도 : 9.372818311874106
보간법 전 일반 속도 표준편차: 3.188774981913939
보간법 전 저속 제거 평균 속도 : 9.372818311874106
보간법 전 저속 제거 속도 표준편차: 3.188774981913939
-
보간법 후 일반 평균 속도 : 10.043791450979361
보간법 후 일반 속도 표준편차: 2.4837009494147684
보간법 후 저속 제거 평균 속도 : 10.043791450979361
보간법 후 저속 제거 속도 표준편차: 2.4837009494147684
보간법 전 일반 평균 속도 : 8.860000000000001
보간법 전 일반 속도 표준편차: 1.4070373290383746
보간법 전 저속 제거 평균 속도 : 8.860000000000001
보간법 전 저속 제거 속도 표준편차: 1.4070373290383746
-
보간법 후 일반 평균 속도 : 8.973051444242168
보간법 후 일반 속도 표준편차: 2.2335699694535394
보간법 후 저속 제거 평균 속도 : 8.973051444242168
보간법 후 저속 제거 속도 표준편차: 2.2335699694535394


In [67]:
mean_speed_df = pd.DataFrame(mean_speed_dict)
list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage)

['AKA_BHUM_9V7282_2022_7', 'AKA_BHUM_9V7282_2022_8']

In [68]:
high_speed_path_data =[]
low_speed_path_data =[]
for i in AKA_BHUM_voyages_list:
    i = i.split("processed")[0]
    if i in list(mean_speed_df[mean_speed_df["mean_speed"] < 14].voyage):
        i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
        df = pd.read_csv(upper_china_path_dir + i + ".csv")
        low_speed_path_data.append(df)
#     elif i in list(mean_speed_df[mean_speed_df["mean_speed"] >= 14].voyage):
#         i = "interpolated/interpolated_" + i + f'processed_below_3knot_{interpolation_number}'
#         df = pd.read_csv(upper_china_path_dir + i + ".csv")
#         high_speed_path_data.append(df)

In [69]:
all_data = pd.concat(low_speed_path_data)

# Add a new index column representing the row number
all_data['index'] = range(len(all_data))
all_data["index"] = all_data.index
concaternated_df = all_data.groupby('index').mean()
concaternated_df.to_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv", encoding="utf-8 sig",header=True, index=False)

In [70]:
# all_data = pd.concat(high_speed_path_data)

# # Add a new index column representing the row number
# all_data['index'] = range(len(all_data))
# all_data["index"] = all_data.index
# concaternated_df = all_data.groupby('index').mean()
# concaternated_df.to_csv(f"실험/upper_china_cosco_fos_high_speed_below_3_knot_{interpolation_number}_concaternated_df.csv", encoding="utf-8 sig",header=True, index=False)

In [71]:
result_df_dict=  {
    "experiment_no" : [],
    "mean_speed" : [],
    "max_speed" : []
}

# ETA 실험 (interpolation 1000), AKA BHUM 7항차 저속

In [72]:
interpolation_number = 1000

In [73]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.669615e+09,117.794500,38.979000,7.700000
1,1.669616e+09,117.801165,38.974246,8.493696
2,1.669616e+09,117.812740,38.969502,9.285811
3,1.669616e+09,117.829662,38.965861,9.775980
4,1.669616e+09,117.847086,38.962360,10.227929
...,...,...,...,...
995,1.669880e+09,128.905563,34.817402,8.110371
996,1.669881e+09,128.907083,34.827681,8.171886
997,1.669881e+09,128.908842,34.836129,8.244922
998,1.669881e+09,128.910421,34.846361,8.290188


In [74]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

## 저속 (7knot 미만 제거)

### segment

In [75]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [76]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [77]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

11.492015745521227
11.193259819644942
10.877774895002695
10.955789903739165
10.839288492665823
10.644641012859493
10.795270451108196
10.726010472759004
10.60885990903097
10.434456803869079
9.618301174439013
11.497158500675354
10.885240745870531
9.109605346132268
8.145193659855545
7.084058402011233
5.15906126124754
5.448864871070281
6.733543053948064
7.889093612293573


### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [78]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [79]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_{interpolation_number}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [80]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 04:51:19.179098
평균: 0 days 01:56:47.617395169


In [81]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
4,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,118.1269,38.8976,16.8,129.0,2022-11-02 15:12:49.646277,2022-11-02 06:06:35,2022-11-02 05:56:14,under way using engine,1045.025952,2326.738581,2022-11-05 01:36:16.898173,2022-11-05 01:34:31,0 days 00:01:45.898173
264,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9472,34.2824,7.2,59.0,2022-11-05 01:41:07.975263,2022-11-04 16:40:24,2022-11-04 04:32:12,under way using engine,110.492280,204.631195,2022-11-05 01:36:26.551629,2022-11-05 01:34:31,0 days 00:01:55.551629
21,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,118.7925,38.7770,11.9,99.0,2022-11-02 17:23:57.155716,2022-11-02 08:19:43,2022-11-02 08:14:16,under way using engine,986.527238,1704.723341,2022-11-05 01:36:57.650746,2022-11-05 01:34:31,0 days 00:02:26.650746
263,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9361,34.2779,7.3,59.0,2022-11-05 01:36:05.224548,2022-11-04 16:35:23,2022-11-04 04:32:12,under way using engine,111.574020,205.132002,2022-11-05 01:31:25.551629,2022-11-05 01:34:31,0 days 00:03:05.448371
64,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,121.0239,38.5067,11.4,114.0,2022-11-03 02:29:03.864811,2022-11-02 17:28:44,2022-11-02 05:14:14,under way using engine,797.848008,1360.631328,2022-11-05 01:38:00.824165,2022-11-05 01:34:31,0 days 00:03:29.824165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.3708,35.1735,12.9,150.0,2022-11-04 04:59:29.903426,2022-11-03 19:58:36,2022-11-02 07:44:12,under way using engine,314.649574,563.597671,2022-11-04 21:02:51.820902,2022-11-05 01:34:31,0 days 04:31:39.179098
99,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.3621,35.1887,12.9,151.0,2022-11-04 04:54:26.719729,2022-11-03 19:53:56,2022-11-02 07:44:12,under way using engine,315.535855,565.185170,2022-11-04 20:58:11.820902,2022-11-05 01:34:31,0 days 04:36:19.179098
98,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.3527,35.2048,12.7,148.0,2022-11-04 04:49:24.626738,2022-11-03 19:48:56,2022-11-02 07:44:12,under way using engine,316.500341,562.341018,2022-11-04 20:53:11.820902,2022-11-05 01:34:31,0 days 04:41:19.179098
97,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.3421,35.2213,13.0,148.0,2022-11-04 04:44:21.288788,2022-11-03 19:43:44,2022-11-02 07:44:12,under way using engine,317.585827,571.253718,2022-11-04 20:47:59.820902,2022-11-05 01:34:31,0 days 04:46:31.179098


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [82]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [83]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_{interpolation_number}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [84]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 04:51:19.179098
평균: 0 days 01:53:11.506819881


In [85]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
237,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6501,34.1865,6.9,68.0,2022-11-04 23:24:53.599991,2022-11-04 14:24:23,2022-11-04 01:38:11,under way using engine,137.996749,261.750760,2022-11-05 01:34:18.458052,2022-11-05 01:34:31,0 days 00:00:12.541948
183,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.0599,34.0799,6.5,79.0,2022-11-04 18:52:18.020141,2022-11-04 09:48:24,2022-11-02 07:44:12,under way using engine,189.127328,372.171594,2022-11-05 01:35:13.394342,2022-11-05 01:34:31,0 days 00:00:42.394342
182,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.0555,34.0794,6.5,79.0,2022-11-04 18:47:16.806349,2022-11-04 09:46:24,2022-11-02 07:44:12,under way using engine,189.500415,372.905768,2022-11-05 01:33:13.394342,2022-11-05 01:34:31,0 days 00:01:17.605658
4,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,118.1269,38.8976,16.8,129.0,2022-11-02 15:12:49.646277,2022-11-02 06:06:35,2022-11-02 05:56:14,under way using engine,1045.025952,2326.738581,2022-11-05 01:36:16.898173,2022-11-05 01:34:31,0 days 00:01:45.898173
264,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9472,34.2824,7.2,59.0,2022-11-05 01:41:07.975263,2022-11-04 16:40:24,2022-11-04 04:32:12,under way using engine,110.492280,204.631195,2022-11-05 01:36:26.551629,2022-11-05 01:34:31,0 days 00:01:55.551629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.3708,35.1735,12.9,150.0,2022-11-04 04:59:29.903426,2022-11-03 19:58:36,2022-11-02 07:44:12,under way using engine,314.649574,563.597671,2022-11-04 21:02:51.820902,2022-11-05 01:34:31,0 days 04:31:39.179098
99,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.3621,35.1887,12.9,151.0,2022-11-04 04:54:26.719729,2022-11-03 19:53:56,2022-11-02 07:44:12,under way using engine,315.535855,565.185170,2022-11-04 20:58:11.820902,2022-11-05 01:34:31,0 days 04:36:19.179098
98,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.3527,35.2048,12.7,148.0,2022-11-04 04:49:24.626738,2022-11-03 19:48:56,2022-11-02 07:44:12,under way using engine,316.500341,562.341018,2022-11-04 20:53:11.820902,2022-11-05 01:34:31,0 days 04:41:19.179098
97,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.3421,35.2213,13.0,148.0,2022-11-04 04:44:21.288788,2022-11-03 19:43:44,2022-11-02 07:44:12,under way using engine,317.585827,571.253718,2022-11-04 20:47:59.820902,2022-11-05 01:34:31,0 days 04:46:31.179098


# ETA 실험 (interpolation 2000), AKA BHUM 7항차 저속

In [86]:
interpolation_number = 2000

In [87]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.669615e+09,117.794500,38.979000,7.700000
1,1.669616e+09,117.797831,38.976624,8.096649
2,1.669616e+09,117.801162,38.974248,8.493299
3,1.669616e+09,117.806255,38.971876,8.889381
4,1.669616e+09,117.812727,38.969507,9.285019
...,...,...,...,...
1995,1.669881e+09,128.908844,34.836140,8.245037
1996,1.669881e+09,128.909670,34.841311,8.302568
1997,1.669881e+09,128.910421,34.846366,8.290093
1998,1.669881e+09,128.911086,34.851283,8.195046


In [88]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

## 저속 (7knot 미만 제거)

### segment

In [89]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [90]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [91]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

11.510774790697194
11.193040764625756
10.87735098839394
10.956954183217517
10.836808796524446
10.644880342419835
10.795392819979439
10.725863961298613
10.608772002154728
10.434400382854081
9.617619400176455
11.49701819557311
10.890544842774943
9.109811905566389
8.149039034438836
7.089983699947414
5.1584166840164025
5.448163098924921
6.721699380390115
7.888603268710094


### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [92]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [93]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_{interpolation_number}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [94]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 04:13:16.915045
평균: 0 days 01:40:27.041994539


In [95]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
16,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,118.6603,38.7941,12.0,101.0,2022-11-02 16:48:39.688732,2022-11-02 07:48:13,2022-11-02 07:20:12,under way using engine,997.854785,1729.557899,2022-11-05 01:34:25.833786,2022-11-05 01:34:31,0 days 00:00:05.166214
17,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,118.6646,38.7935,12.0,100.0,2022-11-02 16:53:39.852756,2022-11-02 07:49:18,2022-11-02 07:50:13,under way using engine,997.484387,1728.915896,2022-11-05 01:35:30.833786,2022-11-05 01:34:31,0 days 00:00:59.833786
236,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6385,34.1844,7.0,71.0,2022-11-04 23:19:48.866175,2022-11-04 14:19:22,2022-11-04 01:38:11,under way using engine,138.969866,261.416107,2022-11-05 01:35:39.442555,2022-11-05 01:34:31,0 days 00:01:08.442555
62,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,120.9088,38.5490,11.5,100.0,2022-11-03 02:03:49.740220,2022-11-02 16:58:34,2022-11-02 04:57:00,under way using engine,808.903883,1382.734303,2022-11-05 01:32:54.613804,2022-11-05 01:34:31,0 days 00:01:36.386196
38,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,120.0154,38.6286,11.1,93.0,2022-11-02 22:21:46.739956,2022-11-02 13:17:35,2022-11-02 01:20:10,under way using engine,882.632802,1495.991873,2022-11-05 01:32:48.508669,2022-11-05 01:34:31,0 days 00:01:42.491331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4097,35.1087,12.4,145.0,2022-11-04 05:19:43.093566,2022-11-03 20:19:06,2022-11-02 07:44:12,under way using engine,310.800535,545.995500,2022-11-04 21:36:04.084955,2022-11-05 01:34:31,0 days 03:58:26.915045
103,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4000,35.1233,12.6,150.0,2022-11-04 05:14:38.376056,2022-11-03 20:14:16,2022-11-02 07:44:12,under way using engine,311.739382,551.733145,2022-11-04 21:31:14.084955,2022-11-05 01:34:31,0 days 04:03:16.915045
139,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.5664,34.5153,14.9,180.0,2022-11-04 08:16:20.520698,2022-11-03 23:15:56,2022-11-02 07:44:12,under way using engine,301.483321,597.000507,2022-11-04 21:31:03.326581,2022-11-05 01:34:31,0 days 04:03:27.673419
102,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.3902,35.1405,12.8,150.0,2022-11-04 05:09:35.685414,2022-11-03 20:08:56,2022-11-02 07:44:12,under way using engine,312.707561,557.826634,2022-11-04 21:25:54.084955,2022-11-05 01:34:31,0 days 04:08:36.915045


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [96]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [97]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_{interpolation_number}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [98]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 04:13:16.915045
평균: 0 days 01:39:28.202920130


In [99]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
16,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,118.6603,38.7941,12.0,101.0,2022-11-02 16:48:39.688732,2022-11-02 07:48:13,2022-11-02 07:20:12,under way using engine,997.854785,1729.557899,2022-11-05 01:34:25.833786,2022-11-05 01:34:31,0 days 00:00:05.166214
17,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,118.6646,38.7935,12.0,100.0,2022-11-02 16:53:39.852756,2022-11-02 07:49:18,2022-11-02 07:50:13,under way using engine,997.484387,1728.915896,2022-11-05 01:35:30.833786,2022-11-05 01:34:31,0 days 00:00:59.833786
236,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6385,34.1844,7.0,71.0,2022-11-04 23:19:48.866175,2022-11-04 14:19:22,2022-11-04 01:38:11,under way using engine,138.969866,261.416107,2022-11-05 01:35:39.442555,2022-11-05 01:34:31,0 days 00:01:08.442555
181,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.0455,34.0785,6.5,79.0,2022-11-04 18:42:12.744927,2022-11-04 09:41:44,2022-11-02 07:44:12,under way using engine,190.334991,374.548078,2022-11-05 01:35:49.127166,2022-11-05 01:34:31,0 days 00:01:18.127166
62,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,120.9088,38.5490,11.5,100.0,2022-11-03 02:03:49.740220,2022-11-02 16:58:34,2022-11-02 04:57:00,under way using engine,808.903883,1382.734303,2022-11-05 01:32:54.613804,2022-11-05 01:34:31,0 days 00:01:36.386196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4000,35.1233,12.6,150.0,2022-11-04 05:14:38.376056,2022-11-03 20:14:16,2022-11-02 07:44:12,under way using engine,311.739382,551.733145,2022-11-04 21:31:14.084955,2022-11-05 01:34:31,0 days 04:03:16.915045
139,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.5664,34.5153,14.9,180.0,2022-11-04 08:16:20.520698,2022-11-03 23:15:56,2022-11-02 07:44:12,under way using engine,301.483321,597.000507,2022-11-04 21:31:03.326581,2022-11-05 01:34:31,0 days 04:03:27.673419
187,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.1122,34.0849,6.5,76.0,2022-11-04 19:12:29.339851,2022-11-04 10:12:14,2022-11-02 07:44:12,under way using engine,184.760482,363.578357,2022-11-04 21:28:31.442555,2022-11-05 01:34:31,0 days 04:05:59.557445
102,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.3902,35.1405,12.8,150.0,2022-11-04 05:09:35.685414,2022-11-03 20:08:56,2022-11-02 07:44:12,under way using engine,312.707561,557.826634,2022-11-04 21:25:54.084955,2022-11-05 01:34:31,0 days 04:08:36.915045


# ETA 실험 (interpolation 2500), AKA BHUM 7항차 저속

In [100]:
interpolation_number = 2500

In [101]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.669615e+09,117.794500,38.979000,7.700000
1,1.669616e+09,117.797164,38.977100,8.017288
2,1.669616e+09,117.799829,38.975199,8.334576
3,1.669616e+09,117.802493,38.973299,8.651863
4,1.669616e+09,117.807547,38.971403,8.968382
...,...,...,...,...
2495,1.669881e+09,128.909505,34.840279,8.291080
2496,1.669881e+09,128.910156,34.844401,8.328088
2497,1.669881e+09,128.910687,34.848334,8.252059
2498,1.669881e+09,128.911219,34.852267,8.176029


In [102]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

## 저속 (7knot 미만 제거)

### segment

In [103]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [104]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [105]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

11.514432600185957
11.193028109496021
10.87708668757587
10.957266646998763
10.836294699070264
10.644928882715002
10.795419918307621
10.725834676594939
10.608754431332516
10.434300945323583
9.617677813593884
11.496875744213641
10.890649437785932
9.109823953575557
8.149840413709509
7.090938191018805
5.158310763123649
5.447953680749357
6.71934444158048
7.888524341606094


### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [106]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [107]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_{interpolation_number}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [108]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 04:06:05.925523
평균: 0 days 01:38:28.728328450


In [109]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
61,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,120.8895,38.5518,11.4,91.0,2022-11-03 01:58:47.061573,2022-11-02 16:53:47,2022-11-02 04:57:00,under way using engine,810.540483,1382.276778,2022-11-05 01:33:06.134192,2022-11-05 01:34:31,0 days 00:01:24.865808
262,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9255,34.2735,7.4,60.0,2022-11-05 01:31:02.428314,2022-11-04 16:30:34,2022-11-04 04:14:10,under way using engine,112.615917,205.608677,2022-11-05 01:32:40.438912,2022-11-05 01:34:31,0 days 00:01:50.561088
236,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6385,34.1844,7.0,71.0,2022-11-04 23:19:48.866175,2022-11-04 14:19:22,2022-11-04 01:38:11,under way using engine,138.969866,261.416107,2022-11-05 01:36:53.526732,2022-11-05 01:34:31,0 days 00:02:22.526732
82,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,121.6580,38.2797,9.8,101.0,2022-11-03 05:35:53.534272,2022-11-02 20:32:08,2022-11-02 07:44:12,under way using engine,737.171981,1238.437552,2022-11-05 01:37:13.366187,2022-11-05 01:34:31,0 days 00:02:42.366187
263,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9361,34.2779,7.3,59.0,2022-11-05 01:36:05.224548,2022-11-04 16:35:23,2022-11-04 04:32:12,under way using engine,111.574020,205.132002,2022-11-05 01:37:29.438912,2022-11-05 01:34:31,0 days 00:02:58.438912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4097,35.1087,12.4,145.0,2022-11-04 05:19:43.093566,2022-11-03 20:19:06,2022-11-02 07:44:12,under way using engine,310.800535,545.995500,2022-11-04 21:38:35.074477,2022-11-05 01:34:31,0 days 03:55:55.925523
140,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.5674,34.4923,16.5,175.0,2022-11-04 08:21:23.318549,2022-11-03 23:21:07,2022-11-02 07:44:12,under way using engine,301.872964,659.184072,2022-11-04 21:38:24.600414,2022-11-05 01:34:31,0 days 03:56:06.399586
150,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.9837,34.0632,8.4,88.0,2022-11-04 11:58:25.435480,2022-11-04 02:56:13,2022-11-02 07:44:12,under way using engine,278.663426,482.472950,2022-11-04 21:38:07.093265,2022-11-05 01:34:31,0 days 03:56:23.906735
103,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4000,35.1233,12.6,150.0,2022-11-04 05:14:38.376056,2022-11-03 20:14:16,2022-11-02 07:44:12,under way using engine,311.739382,551.733145,2022-11-04 21:33:45.074477,2022-11-05 01:34:31,0 days 04:00:45.925523


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [110]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [111]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_{interpolation_number}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [112]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 04:06:05.925523
평균: 0 days 01:37:58.273564750


In [113]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
61,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,120.8895,38.5518,11.4,91.0,2022-11-03 01:58:47.061573,2022-11-02 16:53:47,2022-11-02 04:57:00,under way using engine,810.540483,1382.276778,2022-11-05 01:33:06.134192,2022-11-05 01:34:31,0 days 00:01:24.865808
262,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9255,34.2735,7.4,60.0,2022-11-05 01:31:02.428314,2022-11-04 16:30:34,2022-11-04 04:14:10,under way using engine,112.615917,205.608677,2022-11-05 01:32:40.438912,2022-11-05 01:34:31,0 days 00:01:50.561088
236,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6385,34.1844,7.0,71.0,2022-11-04 23:19:48.866175,2022-11-04 14:19:22,2022-11-04 01:38:11,under way using engine,138.969866,261.416107,2022-11-05 01:36:53.526732,2022-11-05 01:34:31,0 days 00:02:22.526732
235,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6274,34.1819,6.7,64.0,2022-11-04 23:14:46.584631,2022-11-04 14:14:22,2022-11-04 01:38:11,under way using engine,139.937034,270.161194,2022-11-05 01:31:53.526732,2022-11-05 01:34:31,0 days 00:02:37.473268
180,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.0338,34.0775,6.5,79.0,2022-11-04 18:37:09.715818,2022-11-04 09:36:22,2022-11-02 07:44:12,under way using engine,191.309380,376.465514,2022-11-05 01:31:52.483541,2022-11-05 01:34:31,0 days 00:02:38.516459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.9837,34.0632,8.4,88.0,2022-11-04 11:58:25.435480,2022-11-04 02:56:13,2022-11-02 07:44:12,under way using engine,278.663426,482.472950,2022-11-04 21:38:07.093265,2022-11-05 01:34:31,0 days 03:56:23.906735
103,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4000,35.1233,12.6,150.0,2022-11-04 05:14:38.376056,2022-11-03 20:14:16,2022-11-02 07:44:12,under way using engine,311.739382,551.733145,2022-11-04 21:33:45.074477,2022-11-05 01:34:31,0 days 04:00:45.925523
188,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.1193,34.0860,6.4,76.0,2022-11-04 19:17:32.215013,2022-11-04 10:15:34,2022-11-02 07:44:12,under way using engine,184.144127,366.048666,2022-11-04 21:33:05.526732,2022-11-05 01:34:31,0 days 04:01:25.473268
187,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.1122,34.0849,6.5,76.0,2022-11-04 19:12:29.339851,2022-11-04 10:12:14,2022-11-02 07:44:12,under way using engine,184.760482,363.578357,2022-11-04 21:29:45.526732,2022-11-05 01:34:31,0 days 04:04:45.473268


# ETA 실험 (interpolation 3000), AKA BHUM 7항차 저속

In [114]:
interpolation_number = 3000

In [115]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.669615e+09,117.794500,38.979000,7.700000
1,1.669616e+09,117.796720,38.977416,7.964389
2,1.669616e+09,117.798940,38.975833,8.228778
3,1.669616e+09,117.801160,38.974249,8.493167
4,1.669616e+09,117.804094,38.972667,8.757326
...,...,...,...,...
2995,1.669881e+09,128.909945,34.843038,8.321771
2996,1.669881e+09,128.910422,34.846368,8.290061
2997,1.669881e+09,128.910864,34.849645,8.226707
2998,1.669881e+09,128.911307,34.852923,8.163354


In [116]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

## 저속 (7knot 미만 제거)

### segment

In [117]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [118]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [119]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

11.51795313109476
11.192997866179608
10.87706046091978
10.957434623008666
10.835915214345075
10.644960214599367
10.795436611554955
10.725815156714125
10.608742719404006
10.43438215874966
9.617539996555088
11.49681802065187
10.89126519301199
9.1098249251098
8.150308170789094
7.091988332848168
5.158196697960444
5.447897372396346
6.7183238215813885
7.888414589495951


### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [120]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [121]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_{interpolation_number}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [122]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 04:04:27.803001
평균: 0 days 01:37:52.042216591


In [123]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
262,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9255,34.2735,7.4,60.0,2022-11-05 01:31:02.428314,2022-11-04 16:30:34,2022-11-04 04:14:10,under way using engine,112.615917,205.608677,2022-11-05 01:33:18.901317,2022-11-05 01:34:31,0 days 00:01:12.098683
61,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,120.8895,38.5518,11.4,91.0,2022-11-03 01:58:47.061573,2022-11-02 16:53:47,2022-11-02 04:57:00,under way using engine,810.540483,1382.276778,2022-11-05 01:36:20.485513,2022-11-05 01:34:31,0 days 00:01:49.485513
37,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.9760,38.6320,11.1,94.0,2022-11-02 22:11:42.950247,2022-11-02 13:07:34,2022-11-02 12:56:10,under way using engine,885.900279,1501.529984,2022-11-05 01:31:29.768188,2022-11-05 01:34:31,0 days 00:03:01.231812
236,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6385,34.1844,7.0,71.0,2022-11-04 23:19:48.866175,2022-11-04 14:19:22,2022-11-04 01:38:11,under way using engine,138.969866,261.416107,2022-11-05 01:37:42.283913,2022-11-05 01:34:31,0 days 00:03:11.283913
263,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9361,34.2779,7.3,59.0,2022-11-05 01:36:05.224548,2022-11-04 16:35:23,2022-11-04 04:32:12,under way using engine,111.574020,205.132002,2022-11-05 01:38:07.901317,2022-11-05 01:34:31,0 days 00:03:36.901317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4097,35.1087,12.4,145.0,2022-11-04 05:19:43.093566,2022-11-03 20:19:06,2022-11-02 07:44:12,under way using engine,310.800535,545.995500,2022-11-04 21:40:13.196999,2022-11-05 01:34:31,0 days 03:54:17.803001
140,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.5674,34.4923,16.5,175.0,2022-11-04 08:21:23.318549,2022-11-03 23:21:07,2022-11-02 07:44:12,under way using engine,301.872964,659.184072,2022-11-04 21:39:50.395696,2022-11-05 01:34:31,0 days 03:54:40.604304
150,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.9837,34.0632,8.4,88.0,2022-11-04 11:58:25.435480,2022-11-04 02:56:13,2022-11-02 07:44:12,under way using engine,278.663426,482.472950,2022-11-04 21:39:17.493405,2022-11-05 01:34:31,0 days 03:55:13.506595
103,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4000,35.1233,12.6,150.0,2022-11-04 05:14:38.376056,2022-11-03 20:14:16,2022-11-02 07:44:12,under way using engine,311.739382,551.733145,2022-11-04 21:35:23.196999,2022-11-05 01:34:31,0 days 03:59:07.803001


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [124]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [125]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_{interpolation_number}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [126]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 04:04:27.803001
평균: 0 days 01:37:25.740725841


In [127]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
262,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9255,34.2735,7.4,60.0,2022-11-05 01:31:02.428314,2022-11-04 16:30:34,2022-11-04 04:14:10,under way using engine,112.615917,205.608677,2022-11-05 01:33:18.901317,2022-11-05 01:34:31,0 days 00:01:12.098683
180,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.0338,34.0775,6.5,79.0,2022-11-04 18:37:09.715818,2022-11-04 09:36:22,2022-11-02 07:44:12,under way using engine,191.309380,376.465514,2022-11-05 01:32:47.629711,2022-11-05 01:34:31,0 days 00:01:43.370289
235,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6274,34.1819,6.7,64.0,2022-11-04 23:14:46.584631,2022-11-04 14:14:22,2022-11-04 01:38:11,under way using engine,139.937034,270.161194,2022-11-05 01:32:42.283913,2022-11-05 01:34:31,0 days 00:01:48.716087
61,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,120.8895,38.5518,11.4,91.0,2022-11-03 01:58:47.061573,2022-11-02 16:53:47,2022-11-02 04:57:00,under way using engine,810.540483,1382.276778,2022-11-05 01:36:20.485513,2022-11-05 01:34:31,0 days 00:01:49.485513
37,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.9760,38.6320,11.1,94.0,2022-11-02 22:11:42.950247,2022-11-02 13:07:34,2022-11-02 12:56:10,under way using engine,885.900279,1501.529984,2022-11-05 01:31:29.768188,2022-11-05 01:34:31,0 days 00:03:01.231812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.9837,34.0632,8.4,88.0,2022-11-04 11:58:25.435480,2022-11-04 02:56:13,2022-11-02 07:44:12,under way using engine,278.663426,482.472950,2022-11-04 21:39:17.493405,2022-11-05 01:34:31,0 days 03:55:13.506595
103,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4000,35.1233,12.6,150.0,2022-11-04 05:14:38.376056,2022-11-03 20:14:16,2022-11-02 07:44:12,under way using engine,311.739382,551.733145,2022-11-04 21:35:23.196999,2022-11-05 01:34:31,0 days 03:59:07.803001
188,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.1193,34.0860,6.4,76.0,2022-11-04 19:17:32.215013,2022-11-04 10:15:34,2022-11-02 07:44:12,under way using engine,184.144127,366.048666,2022-11-04 21:33:54.283913,2022-11-05 01:34:31,0 days 04:00:36.716087
187,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.1122,34.0849,6.5,76.0,2022-11-04 19:12:29.339851,2022-11-04 10:12:14,2022-11-02 07:44:12,under way using engine,184.760482,363.578357,2022-11-04 21:30:34.283913,2022-11-05 01:34:31,0 days 04:03:56.716087


# ETA 실험 (interpolation 4000), AKA BHUM 7항차 저속

In [128]:
interpolation_number = 4000

In [129]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.669615e+09,117.794500,38.979000,7.700000
1,1.669615e+09,117.796165,38.977812,7.898275
2,1.669616e+09,117.797830,38.976625,8.096550
3,1.669616e+09,117.799495,38.975437,8.294825
4,1.669616e+09,117.801160,38.974249,8.493100
...,...,...,...,...
3995,1.669881e+09,128.910422,34.846369,8.290045
3996,1.669881e+09,128.910754,34.848827,8.242534
3997,1.669881e+09,128.911086,34.851284,8.195023
3998,1.669881e+09,128.911418,34.853742,8.147511


In [130]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

## 저속 (7knot 미만 제거)

### segment

In [131]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [132]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [133]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

11.520387329715373
11.192987927149474
10.876998308826238
10.957697809808984
10.835477802991168
10.644999703215026
10.795457088786783
10.725790760523951
10.608728081689923
10.434371529545356
9.617398266877755
11.49677900143377
10.891735629375464
9.10989107463032
8.150946543630901
7.093063043492451
5.158075917913961
5.447796938665947
6.716279640784401
7.888366033979094


### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [134]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [135]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_{interpolation_number}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [136]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 04:02:23.303773
평균: 0 days 01:37:21.127528056


In [137]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
262,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9255,34.2735,7.4,60.0,2022-11-05 01:31:02.428314,2022-11-04 16:30:34,2022-11-04 04:14:10,under way using engine,112.615917,205.608677,2022-11-05 01:34:07.830169,2022-11-05 01:34:31,0 days 00:00:23.169831
37,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.9760,38.6320,11.1,94.0,2022-11-02 22:11:42.950247,2022-11-02 13:07:34,2022-11-02 12:56:10,under way using engine,885.900279,1501.529984,2022-11-05 01:35:48.865789,2022-11-05 01:34:31,0 days 00:01:17.865789
236,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6385,34.1844,7.0,71.0,2022-11-04 23:19:48.866175,2022-11-04 14:19:22,2022-11-04 01:38:11,under way using engine,138.969866,261.416107,2022-11-05 01:38:44.034865,2022-11-05 01:34:31,0 days 00:04:13.034865
263,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9361,34.2779,7.3,59.0,2022-11-05 01:36:05.224548,2022-11-04 16:35:23,2022-11-04 04:32:12,under way using engine,111.574020,205.132002,2022-11-05 01:38:56.830169,2022-11-05 01:34:31,0 days 00:04:25.830169
60,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,120.8479,38.5566,11.5,91.0,2022-11-03 01:43:38.286928,2022-11-02 16:43:24,2022-11-02 04:38:10,under way using engine,814.007071,1391.457653,2022-11-05 01:30:01.955449,2022-11-05 01:34:31,0 days 00:04:29.044551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4097,35.1087,12.4,145.0,2022-11-04 05:19:43.093566,2022-11-03 20:19:06,2022-11-02 07:44:12,under way using engine,310.800535,545.995500,2022-11-04 21:42:17.696227,2022-11-05 01:34:31,0 days 03:52:13.303773
140,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.5674,34.4923,16.5,175.0,2022-11-04 08:21:23.318549,2022-11-03 23:21:07,2022-11-02 07:44:12,under way using engine,301.872964,659.184072,2022-11-04 21:41:38.149000,2022-11-05 01:34:31,0 days 03:52:52.851000
150,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.9837,34.0632,8.4,88.0,2022-11-04 11:58:25.435480,2022-11-04 02:56:13,2022-11-02 07:44:12,under way using engine,278.663426,482.472950,2022-11-04 21:40:46.633260,2022-11-05 01:34:31,0 days 03:53:44.366740
103,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4000,35.1233,12.6,150.0,2022-11-04 05:14:38.376056,2022-11-03 20:14:16,2022-11-02 07:44:12,under way using engine,311.739382,551.733145,2022-11-04 21:37:27.696227,2022-11-05 01:34:31,0 days 03:57:03.303773


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [138]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [139]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_{interpolation_number}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [140]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 04:02:54.965135
평균: 0 days 01:36:54.815113510


In [141]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
262,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9255,34.2735,7.4,60.0,2022-11-05 01:31:02.428314,2022-11-04 16:30:34,2022-11-04 04:14:10,under way using engine,112.615917,205.608677,2022-11-05 01:34:07.830169,2022-11-05 01:34:31,0 days 00:00:23.169831
180,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.0338,34.0775,6.5,79.0,2022-11-04 18:37:09.715818,2022-11-04 09:36:22,2022-11-02 07:44:12,under way using engine,191.309380,376.465514,2022-11-05 01:33:58.003691,2022-11-05 01:34:31,0 days 00:00:32.996309
235,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6274,34.1819,6.7,64.0,2022-11-04 23:14:46.584631,2022-11-04 14:14:22,2022-11-04 01:38:11,under way using engine,139.937034,270.161194,2022-11-05 01:33:44.034865,2022-11-05 01:34:31,0 days 00:00:46.965135
37,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.9760,38.6320,11.1,94.0,2022-11-02 22:11:42.950247,2022-11-02 13:07:34,2022-11-02 12:56:10,under way using engine,885.900279,1501.529984,2022-11-05 01:35:48.865789,2022-11-05 01:34:31,0 days 00:01:17.865789
236,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6385,34.1844,7.0,71.0,2022-11-04 23:19:48.866175,2022-11-04 14:19:22,2022-11-04 01:38:11,under way using engine,138.969866,261.416107,2022-11-05 01:38:44.034865,2022-11-05 01:34:31,0 days 00:04:13.034865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.9837,34.0632,8.4,88.0,2022-11-04 11:58:25.435480,2022-11-04 02:56:13,2022-11-02 07:44:12,under way using engine,278.663426,482.472950,2022-11-04 21:40:46.633260,2022-11-05 01:34:31,0 days 03:53:44.366740
103,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4000,35.1233,12.6,150.0,2022-11-04 05:14:38.376056,2022-11-03 20:14:16,2022-11-02 07:44:12,under way using engine,311.739382,551.733145,2022-11-04 21:37:27.696227,2022-11-05 01:34:31,0 days 03:57:03.303773
188,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.1193,34.0860,6.4,76.0,2022-11-04 19:17:32.215013,2022-11-04 10:15:34,2022-11-02 07:44:12,under way using engine,184.144127,366.048666,2022-11-04 21:34:56.034865,2022-11-05 01:34:31,0 days 03:59:34.965135
102,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.3902,35.1405,12.8,150.0,2022-11-04 05:09:35.685414,2022-11-03 20:08:56,2022-11-02 07:44:12,under way using engine,312.707561,557.826634,2022-11-04 21:32:07.696227,2022-11-05 01:34:31,0 days 04:02:23.303773


# ETA 실험 (interpolation 5000), AKA BHUM 7항차 저속

In [142]:
interpolation_number = 5000

In [143]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.669615e+09,117.794500,38.979000,7.700000
1,1.669615e+09,117.795832,38.978050,7.858612
2,1.669616e+09,117.797164,38.977100,8.017224
3,1.669616e+09,117.798496,38.976150,8.175836
4,1.669616e+09,117.799828,38.975200,8.334449
...,...,...,...,...
4995,1.669881e+09,128.910687,34.848335,8.252029
4996,1.669881e+09,128.910953,34.850302,8.214021
4997,1.669881e+09,128.911219,34.852268,8.176014
4998,1.669881e+09,128.911484,34.854234,8.138007


In [144]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

## 저속 (7knot 미만 제거)

### segment

In [145]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [146]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [147]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

11.522032962566767
11.192983967448304
10.876974976879406
10.957808243776999
10.835187352534852
10.64502325323558
10.795470366527656
10.725776124761946
10.60871930023272
10.434366194017992
9.617347644140175
11.496706387782222
10.892003936397451
9.109923239996009
8.151314742525063
7.0936734336307055
5.157999969952467
5.44776188764543
6.7153207944511
7.888337785651469


### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [148]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [149]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_{interpolation_number}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [150]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 04:01:10.044358
평균: 0 days 01:37:03.167174211


In [151]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
262,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9255,34.2735,7.4,60.0,2022-11-05 01:31:02.428314,2022-11-04 16:30:34,2022-11-04 04:14:10,under way using engine,112.615917,205.608677,2022-11-05 01:34:35.237440,2022-11-05 01:34:31,0 days 00:00:04.237440
60,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,120.8479,38.5566,11.5,91.0,2022-11-03 01:43:38.286928,2022-11-02 16:43:24,2022-11-02 04:38:10,under way using engine,814.007071,1391.457653,2022-11-05 01:32:26.755726,2022-11-05 01:34:31,0 days 00:02:04.244274
36,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.9483,38.6342,11.1,94.0,2022-11-02 22:01:34.353827,2022-11-02 13:00:34,2022-11-02 12:56:10,under way using engine,888.189640,1505.410268,2022-11-05 01:31:22.670586,2022-11-05 01:34:31,0 days 00:03:08.329414
37,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.9760,38.6320,11.1,94.0,2022-11-02 22:11:42.950247,2022-11-02 13:07:34,2022-11-02 12:56:10,under way using engine,885.900279,1501.529984,2022-11-05 01:38:22.670586,2022-11-05 01:34:31,0 days 00:03:51.670586
81,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,121.6065,38.2938,10.0,102.0,2022-11-03 05:20:40.775978,2022-11-02 20:16:25,2022-11-02 07:44:12,under way using engine,741.880266,1245.460598,2022-11-05 01:30:25.169737,2022-11-05 01:34:31,0 days 00:04:05.830263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4097,35.1087,12.4,145.0,2022-11-04 05:19:43.093566,2022-11-03 20:19:06,2022-11-02 07:44:12,under way using engine,310.800535,545.995500,2022-11-04 21:43:30.955642,2022-11-05 01:34:31,0 days 03:51:00.044358
140,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.5674,34.4923,16.5,175.0,2022-11-04 08:21:23.318549,2022-11-03 23:21:07,2022-11-02 07:44:12,under way using engine,301.872964,659.184072,2022-11-04 21:42:41.463944,2022-11-05 01:34:31,0 days 03:51:49.536056
150,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.9837,34.0632,8.4,88.0,2022-11-04 11:58:25.435480,2022-11-04 02:56:13,2022-11-02 07:44:12,under way using engine,278.663426,482.472950,2022-11-04 21:41:38.589225,2022-11-05 01:34:31,0 days 03:52:52.410775
103,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4000,35.1233,12.6,150.0,2022-11-04 05:14:38.376056,2022-11-03 20:14:16,2022-11-02 07:44:12,under way using engine,311.739382,551.733145,2022-11-04 21:38:40.955642,2022-11-05 01:34:31,0 days 03:55:50.044358


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [152]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [153]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_{interpolation_number}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [154]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 04:02:19.904080
평균: 0 days 01:35:57.489220507


In [155]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
262,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9255,34.2735,7.4,60.0,2022-11-05 01:31:02.428314,2022-11-04 16:30:34,2022-11-04 04:14:10,under way using engine,112.615917,205.608677,2022-11-05 01:34:35.237440,2022-11-05 01:34:31,0 days 00:00:04.237440
180,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.0338,34.0775,6.5,79.0,2022-11-04 18:37:09.715818,2022-11-04 09:36:22,2022-11-02 07:44:12,under way using engine,191.309380,376.465514,2022-11-05 01:34:38.629233,2022-11-05 01:34:31,0 days 00:00:07.629233
235,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6274,34.1819,6.7,64.0,2022-11-04 23:14:46.584631,2022-11-04 14:14:22,2022-11-04 01:38:11,under way using engine,139.937034,270.161194,2022-11-05 01:34:19.095920,2022-11-05 01:34:31,0 days 00:00:11.904080
60,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,120.8479,38.5566,11.5,91.0,2022-11-03 01:43:38.286928,2022-11-02 16:43:24,2022-11-02 04:38:10,under way using engine,814.007071,1391.457653,2022-11-05 01:32:26.755726,2022-11-05 01:34:31,0 days 00:02:04.244274
36,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.9483,38.6342,11.1,94.0,2022-11-02 22:01:34.353827,2022-11-02 13:00:34,2022-11-02 12:56:10,under way using engine,888.189640,1505.410268,2022-11-05 01:31:22.670586,2022-11-05 01:34:31,0 days 00:03:08.329414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.9837,34.0632,8.4,88.0,2022-11-04 11:58:25.435480,2022-11-04 02:56:13,2022-11-02 07:44:12,under way using engine,278.663426,482.472950,2022-11-04 21:41:38.589225,2022-11-05 01:34:31,0 days 03:52:52.410775
103,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4000,35.1233,12.6,150.0,2022-11-04 05:14:38.376056,2022-11-03 20:14:16,2022-11-02 07:44:12,under way using engine,311.739382,551.733145,2022-11-04 21:38:40.955642,2022-11-05 01:34:31,0 days 03:55:50.044358
188,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.1193,34.0860,6.4,76.0,2022-11-04 19:17:32.215013,2022-11-04 10:15:34,2022-11-02 07:44:12,under way using engine,184.144127,366.048666,2022-11-04 21:35:31.095920,2022-11-05 01:34:31,0 days 03:58:59.904080
102,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.3902,35.1405,12.8,150.0,2022-11-04 05:09:35.685414,2022-11-03 20:08:56,2022-11-02 07:44:12,under way using engine,312.707561,557.826634,2022-11-04 21:33:20.955642,2022-11-05 01:34:31,0 days 04:01:10.044358


# ETA 실험 (interpolation 7500), AKA BHUM 7항차 저속

In [156]:
interpolation_number = 7500

In [157]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.669615e+09,117.794500,38.979000,7.700000
1,1.669615e+09,117.795388,38.978367,7.805734
2,1.669616e+09,117.796276,38.977733,7.911469
3,1.669616e+09,117.797164,38.977100,8.017203
4,1.669616e+09,117.798052,38.976467,8.122938
...,...,...,...,...
7495,1.669881e+09,128.911042,34.850957,8.201346
7496,1.669881e+09,128.911219,34.852268,8.176009
7497,1.669881e+09,128.911396,34.853579,8.150673
7498,1.669882e+09,128.911573,34.854889,8.125336


In [158]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

## 저속 (7knot 미만 제거)

### segment

In [159]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [160]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [161]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

11.524448036667609
11.19295460672496
10.876933774395583
10.958013124903742
10.834851852610827
10.645054701686394
10.795487114776927
10.725756612689578
10.608707592989303
10.434353890154807
9.617275544483507
11.496676316028374
10.892369370487666
9.109966473053289
8.151832845796159
7.094520132941012
5.1578976788061395
5.447695251458785
6.713765597854171
7.888213535520908


### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [162]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [163]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_{interpolation_number}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [164]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 03:59:31.604528
평균: 0 days 01:37:17.984879253


In [165]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
36,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.9483,38.6342,11.1,94.0,2022-11-02 22:01:34.353827,2022-11-02 13:00:34,2022-11-02 12:56:10,under way using engine,888.189640,1505.410268,2022-11-05 01:34:49.414305,2022-11-05 01:34:31,0 days 00:00:18.414305
262,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9255,34.2735,7.4,60.0,2022-11-05 01:31:02.428314,2022-11-04 16:30:34,2022-11-04 04:14:10,under way using engine,112.615917,205.608677,2022-11-05 01:35:13.198844,2022-11-05 01:34:31,0 days 00:00:42.198844
60,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,120.8479,38.5566,11.5,91.0,2022-11-03 01:43:38.286928,2022-11-02 16:43:24,2022-11-02 04:38:10,under way using engine,814.007071,1391.457653,2022-11-05 01:35:39.778712,2022-11-05 01:34:31,0 days 00:01:08.778712
81,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,121.6065,38.2938,10.0,102.0,2022-11-03 05:20:40.775978,2022-11-02 20:16:25,2022-11-02 07:44:12,under way using engine,741.880266,1245.460598,2022-11-05 01:33:22.113017,2022-11-05 01:34:31,0 days 00:01:08.886983
261,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9146,34.2690,7.3,59.0,2022-11-05 01:25:59.438198,2022-11-04 16:25:43,2022-11-04 04:14:10,under way using engine,113.686369,209.015615,2022-11-05 01:30:22.198844,2022-11-05 01:34:31,0 days 00:04:08.801156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4097,35.1087,12.4,145.0,2022-11-04 05:19:43.093566,2022-11-03 20:19:06,2022-11-02 07:44:12,under way using engine,310.800535,545.995500,2022-11-04 21:45:09.395472,2022-11-05 01:34:31,0 days 03:49:21.604528
140,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.5674,34.4923,16.5,175.0,2022-11-04 08:21:23.318549,2022-11-03 23:21:07,2022-11-02 07:44:12,under way using engine,301.872964,659.184072,2022-11-04 21:44:06.385057,2022-11-05 01:34:31,0 days 03:50:24.614943
150,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.9837,34.0632,8.4,88.0,2022-11-04 11:58:25.435480,2022-11-04 02:56:13,2022-11-02 07:44:12,under way using engine,278.663426,482.472950,2022-11-04 21:42:48.537514,2022-11-05 01:34:31,0 days 03:51:42.462486
103,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4000,35.1233,12.6,150.0,2022-11-04 05:14:38.376056,2022-11-03 20:14:16,2022-11-02 07:44:12,under way using engine,311.739382,551.733145,2022-11-04 21:40:19.395472,2022-11-05 01:34:31,0 days 03:54:11.604528


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [166]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [167]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_{interpolation_number}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [168]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 04:01:31.677562
평균: 0 days 01:35:59.055857644


In [169]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
36,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.9483,38.6342,11.1,94.0,2022-11-02 22:01:34.353827,2022-11-02 13:00:34,2022-11-02 12:56:10,under way using engine,888.189640,1505.410268,2022-11-05 01:34:49.414305,2022-11-05 01:34:31,0 days 00:00:18.414305
235,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6274,34.1819,6.7,64.0,2022-11-04 23:14:46.584631,2022-11-04 14:14:22,2022-11-04 01:38:11,under way using engine,139.937034,270.161194,2022-11-05 01:35:07.322438,2022-11-05 01:34:31,0 days 00:00:36.322438
262,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9255,34.2735,7.4,60.0,2022-11-05 01:31:02.428314,2022-11-04 16:30:34,2022-11-04 04:14:10,under way using engine,112.615917,205.608677,2022-11-05 01:35:13.198844,2022-11-05 01:34:31,0 days 00:00:42.198844
180,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.0338,34.0775,6.5,79.0,2022-11-04 18:37:09.715818,2022-11-04 09:36:22,2022-11-02 07:44:12,under way using engine,191.309380,376.465514,2022-11-05 01:35:33.479026,2022-11-05 01:34:31,0 days 00:01:02.479026
60,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,120.8479,38.5566,11.5,91.0,2022-11-03 01:43:38.286928,2022-11-02 16:43:24,2022-11-02 04:38:10,under way using engine,814.007071,1391.457653,2022-11-05 01:35:39.778712,2022-11-05 01:34:31,0 days 00:01:08.778712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.9837,34.0632,8.4,88.0,2022-11-04 11:58:25.435480,2022-11-04 02:56:13,2022-11-02 07:44:12,under way using engine,278.663426,482.472950,2022-11-04 21:42:48.537514,2022-11-05 01:34:31,0 days 03:51:42.462486
103,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4000,35.1233,12.6,150.0,2022-11-04 05:14:38.376056,2022-11-03 20:14:16,2022-11-02 07:44:12,under way using engine,311.739382,551.733145,2022-11-04 21:40:19.395472,2022-11-05 01:34:31,0 days 03:54:11.604528
188,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.1193,34.0860,6.4,76.0,2022-11-04 19:17:32.215013,2022-11-04 10:15:34,2022-11-02 07:44:12,under way using engine,184.144127,366.048666,2022-11-04 21:36:19.322438,2022-11-05 01:34:31,0 days 03:58:11.677562
102,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.3902,35.1405,12.8,150.0,2022-11-04 05:09:35.685414,2022-11-03 20:08:56,2022-11-02 07:44:12,under way using engine,312.707561,557.826634,2022-11-04 21:34:59.395472,2022-11-05 01:34:31,0 days 03:59:31.604528


# ETA 실험 (interpolation 9000), AKA BHUM 7항차 저속

In [170]:
interpolation_number = 9000

In [171]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.669615e+09,117.794500,38.979000,7.700000
1,1.669615e+09,117.795240,38.978472,7.788110
2,1.669615e+09,117.795980,38.977944,7.876220
3,1.669616e+09,117.796720,38.977417,7.964330
4,1.669616e+09,117.797460,38.976889,8.052440
...,...,...,...,...
8995,1.669881e+09,128.911160,34.851831,8.184453
8996,1.669881e+09,128.911307,34.852923,8.163340
8997,1.669881e+09,128.911455,34.854016,8.142226
8998,1.669882e+09,128.911602,34.855108,8.121113


In [172]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

## 저속 (7knot 미만 제거)

### segment

In [173]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [174]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [175]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

11.525267561529013
11.192948062431007
10.876921725229716
10.958072849231776
10.834731451781352
10.64506517288829
10.795492966585597
10.725750109243656
10.608703690921745
10.434358645426435
9.617244208962523
11.496655794215517
10.892536709987484
9.109975060455291
8.151991837576452
7.0948206694370715
5.157860450272062
5.447674959936275
6.7132740902668155
7.888199307793515


### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [176]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [177]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_{interpolation_number}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [178]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 03:58:58.609301
평균: 0 days 01:38:35.184024089


In [179]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
81,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,121.6065,38.2938,10.0,102.0,2022-11-03 05:20:40.775978,2022-11-02 20:16:25,2022-11-02 07:44:12,under way using engine,741.880266,1245.460598,2022-11-05 01:34:22.582072,2022-11-05 01:34:31,0 days 00:00:08.417928
262,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9255,34.2735,7.4,60.0,2022-11-05 01:31:02.428314,2022-11-04 16:30:34,2022-11-04 04:14:10,under way using engine,112.615917,205.608677,2022-11-05 01:35:26.245348,2022-11-05 01:34:31,0 days 00:00:55.245348
36,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.9483,38.6342,11.1,94.0,2022-11-02 22:01:34.353827,2022-11-02 13:00:34,2022-11-02 12:56:10,under way using engine,888.189640,1505.410268,2022-11-05 01:36:00.856498,2022-11-05 01:34:31,0 days 00:01:29.856498
60,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,120.8479,38.5566,11.5,91.0,2022-11-03 01:43:38.286928,2022-11-02 16:43:24,2022-11-02 04:38:10,under way using engine,814.007071,1391.457653,2022-11-05 01:36:45.672866,2022-11-05 01:34:31,0 days 00:02:14.672866
261,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9146,34.2690,7.3,59.0,2022-11-05 01:25:59.438198,2022-11-04 16:25:43,2022-11-04 04:14:10,under way using engine,113.686369,209.015615,2022-11-05 01:30:35.245348,2022-11-05 01:34:31,0 days 00:03:55.754652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4097,35.1087,12.4,145.0,2022-11-04 05:19:43.093566,2022-11-03 20:19:06,2022-11-02 07:44:12,under way using engine,310.800535,545.995500,2022-11-04 21:45:42.390699,2022-11-05 01:34:31,0 days 03:48:48.609301
140,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.5674,34.4923,16.5,175.0,2022-11-04 08:21:23.318549,2022-11-03 23:21:07,2022-11-02 07:44:12,under way using engine,301.872964,659.184072,2022-11-04 21:44:35.217572,2022-11-05 01:34:31,0 days 03:49:55.782428
150,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.9837,34.0632,8.4,88.0,2022-11-04 11:58:25.435480,2022-11-04 02:56:13,2022-11-02 07:44:12,under way using engine,278.663426,482.472950,2022-11-04 21:43:12.317814,2022-11-05 01:34:31,0 days 03:51:18.682186
103,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4000,35.1233,12.6,150.0,2022-11-04 05:14:38.376056,2022-11-03 20:14:16,2022-11-02 07:44:12,under way using engine,311.739382,551.733145,2022-11-04 21:40:52.390699,2022-11-05 01:34:31,0 days 03:53:38.609301


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [180]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [181]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_{interpolation_number}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [182]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 04:01:15.220200
평균: 0 days 01:36:46.320734629


In [183]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
81,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,121.6065,38.2938,10.0,102.0,2022-11-03 05:20:40.775978,2022-11-02 20:16:25,2022-11-02 07:44:12,under way using engine,741.880266,1245.460598,2022-11-05 01:34:22.582072,2022-11-05 01:34:31,0 days 00:00:08.417928
235,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6274,34.1819,6.7,64.0,2022-11-04 23:14:46.584631,2022-11-04 14:14:22,2022-11-04 01:38:11,under way using engine,139.937034,270.161194,2022-11-05 01:35:23.779800,2022-11-05 01:34:31,0 days 00:00:52.779800
262,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9255,34.2735,7.4,60.0,2022-11-05 01:31:02.428314,2022-11-04 16:30:34,2022-11-04 04:14:10,under way using engine,112.615917,205.608677,2022-11-05 01:35:26.245348,2022-11-05 01:34:31,0 days 00:00:55.245348
180,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.0338,34.0775,6.5,79.0,2022-11-04 18:37:09.715818,2022-11-04 09:36:22,2022-11-02 07:44:12,under way using engine,191.309380,376.465514,2022-11-05 01:35:52.250595,2022-11-05 01:34:31,0 days 00:01:21.250595
36,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.9483,38.6342,11.1,94.0,2022-11-02 22:01:34.353827,2022-11-02 13:00:34,2022-11-02 12:56:10,under way using engine,888.189640,1505.410268,2022-11-05 01:36:00.856498,2022-11-05 01:34:31,0 days 00:01:29.856498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.1331,34.0882,6.4,76.0,2022-11-04 19:22:35.135832,2022-11-04 10:22:05,2022-11-02 07:44:12,under way using engine,182.943349,363.661714,2022-11-04 21:43:06.779800,2022-11-05 01:34:31,0 days 03:51:24.220200
103,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4000,35.1233,12.6,150.0,2022-11-04 05:14:38.376056,2022-11-03 20:14:16,2022-11-02 07:44:12,under way using engine,311.739382,551.733145,2022-11-04 21:40:52.390699,2022-11-05 01:34:31,0 days 03:53:38.609301
188,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.1193,34.0860,6.4,76.0,2022-11-04 19:17:32.215013,2022-11-04 10:15:34,2022-11-02 07:44:12,under way using engine,184.144127,366.048666,2022-11-04 21:36:35.779800,2022-11-05 01:34:31,0 days 03:57:55.220200
102,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.3902,35.1405,12.8,150.0,2022-11-04 05:09:35.685414,2022-11-03 20:08:56,2022-11-02 07:44:12,under way using engine,312.707561,557.826634,2022-11-04 21:35:32.390699,2022-11-05 01:34:31,0 days 03:58:58.609301


# ETA 실험 (interpolation 10000), AKA BHUM 7항차 저속

In [184]:
interpolation_number = 10000

In [185]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.669615e+09,117.794500,38.979000,7.700000
1,1.669615e+09,117.795166,38.978525,7.779298
2,1.669615e+09,117.795832,38.978050,7.858596
3,1.669616e+09,117.796498,38.977575,7.937894
4,1.669616e+09,117.797164,38.977100,8.017193
...,...,...,...,...
9995,1.669881e+09,128.911219,34.852268,8.176007
9996,1.669881e+09,128.911352,34.853251,8.157005
9997,1.669881e+09,128.911484,34.854234,8.138003
9998,1.669882e+09,128.911617,34.855217,8.119002


In [186]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

## 저속 (7knot 미만 제거)

### segment

In [187]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [188]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [189]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

11.525651713483978
11.192940660157102
10.876914729531247
10.958090148166608
10.834673804919037
10.645070424339172
10.79549563464511
10.725746857629083
10.608701739952984
10.434357795428356
9.617239818190374
11.49665778734069
10.89260573738575
9.109980511538337
8.152094858612907
7.094969654638753
5.157841579210552
5.447664931972969
6.7130297644710115
7.888195245356821


### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [190]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [191]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_{interpolation_number}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [192]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 03:58:42.341169
평균: 0 days 01:38:32.684934629


In [193]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
81,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,121.6065,38.2938,10.0,102.0,2022-11-03 05:20:40.775978,2022-11-02 20:16:25,2022-11-02 07:44:12,under way using engine,741.880266,1245.460598,2022-11-05 01:34:51.872955,2022-11-05 01:34:31,0 days 00:00:20.872955
262,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9255,34.2735,7.4,60.0,2022-11-05 01:31:02.428314,2022-11-04 16:30:34,2022-11-04 04:14:10,under way using engine,112.615917,205.608677,2022-11-05 01:35:32.154777,2022-11-05 01:34:31,0 days 00:01:01.154777
36,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.9483,38.6342,11.1,94.0,2022-11-02 22:01:34.353827,2022-11-02 13:00:34,2022-11-02 12:56:10,under way using engine,888.189640,1505.410268,2022-11-05 01:36:35.794979,2022-11-05 01:34:31,0 days 00:02:04.794979
60,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,120.8479,38.5566,11.5,91.0,2022-11-03 01:43:38.286928,2022-11-02 16:43:24,2022-11-02 04:38:10,under way using engine,814.007071,1391.457653,2022-11-05 01:37:17.731297,2022-11-05 01:34:31,0 days 00:02:46.731297
261,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9146,34.2690,7.3,59.0,2022-11-05 01:25:59.438198,2022-11-04 16:25:43,2022-11-04 04:14:10,under way using engine,113.686369,209.015615,2022-11-05 01:30:41.154777,2022-11-05 01:34:31,0 days 00:03:49.845223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4097,35.1087,12.4,145.0,2022-11-04 05:19:43.093566,2022-11-03 20:19:06,2022-11-02 07:44:12,under way using engine,310.800535,545.995500,2022-11-04 21:45:58.658831,2022-11-05 01:34:31,0 days 03:48:32.341169
140,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.5674,34.4923,16.5,175.0,2022-11-04 08:21:23.318549,2022-11-03 23:21:07,2022-11-02 07:44:12,under way using engine,301.872964,659.184072,2022-11-04 21:44:49.098228,2022-11-05 01:34:31,0 days 03:49:41.901772
150,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.9837,34.0632,8.4,88.0,2022-11-04 11:58:25.435480,2022-11-04 02:56:13,2022-11-02 07:44:12,under way using engine,278.663426,482.472950,2022-11-04 21:43:23.664678,2022-11-05 01:34:31,0 days 03:51:07.335322
103,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4000,35.1233,12.6,150.0,2022-11-04 05:14:38.376056,2022-11-03 20:14:16,2022-11-02 07:44:12,under way using engine,311.739382,551.733145,2022-11-04 21:41:08.658831,2022-11-05 01:34:31,0 days 03:53:22.341169


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [194]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [195]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_{interpolation_number}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [196]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 04:01:07.590395
평균: 0 days 01:36:43.464540446


In [197]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
81,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,121.6065,38.2938,10.0,102.0,2022-11-03 05:20:40.775978,2022-11-02 20:16:25,2022-11-02 07:44:12,under way using engine,741.880266,1245.460598,2022-11-05 01:34:51.872955,2022-11-05 01:34:31,0 days 00:00:20.872955
235,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6274,34.1819,6.7,64.0,2022-11-04 23:14:46.584631,2022-11-04 14:14:22,2022-11-04 01:38:11,under way using engine,139.937034,270.161194,2022-11-05 01:35:31.409605,2022-11-05 01:34:31,0 days 00:01:00.409605
262,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9255,34.2735,7.4,60.0,2022-11-05 01:31:02.428314,2022-11-04 16:30:34,2022-11-04 04:14:10,under way using engine,112.615917,205.608677,2022-11-05 01:35:32.154777,2022-11-05 01:34:31,0 days 00:01:01.154777
180,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.0338,34.0775,6.5,79.0,2022-11-04 18:37:09.715818,2022-11-04 09:36:22,2022-11-02 07:44:12,under way using engine,191.309380,376.465514,2022-11-05 01:36:01.095888,2022-11-05 01:34:31,0 days 00:01:30.095888
36,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.9483,38.6342,11.1,94.0,2022-11-02 22:01:34.353827,2022-11-02 13:00:34,2022-11-02 12:56:10,under way using engine,888.189640,1505.410268,2022-11-05 01:36:35.794979,2022-11-05 01:34:31,0 days 00:02:04.794979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.1331,34.0882,6.4,76.0,2022-11-04 19:22:35.135832,2022-11-04 10:22:05,2022-11-02 07:44:12,under way using engine,182.943349,363.661714,2022-11-04 21:43:14.409605,2022-11-05 01:34:31,0 days 03:51:16.590395
103,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4000,35.1233,12.6,150.0,2022-11-04 05:14:38.376056,2022-11-03 20:14:16,2022-11-02 07:44:12,under way using engine,311.739382,551.733145,2022-11-04 21:41:08.658831,2022-11-05 01:34:31,0 days 03:53:22.341169
188,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.1193,34.0860,6.4,76.0,2022-11-04 19:17:32.215013,2022-11-04 10:15:34,2022-11-02 07:44:12,under way using engine,184.144127,366.048666,2022-11-04 21:36:43.409605,2022-11-05 01:34:31,0 days 03:57:47.590395
102,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.3902,35.1405,12.8,150.0,2022-11-04 05:09:35.685414,2022-11-03 20:08:56,2022-11-02 07:44:12,under way using engine,312.707561,557.826634,2022-11-04 21:35:48.658831,2022-11-05 01:34:31,0 days 03:58:42.341169


# ETA 실험 (interpolation 11000), AKA BHUM 7항차 저속

In [198]:
interpolation_number = 11000

In [199]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.669615e+09,117.794500,38.979000,7.700000
1,1.669615e+09,117.795105,38.978568,7.772089
2,1.669615e+09,117.795711,38.978136,7.844177
3,1.669616e+09,117.796316,38.977705,7.916266
4,1.669616e+09,117.796921,38.977273,7.988354
...,...,...,...,...
10995,1.669881e+09,128.911267,34.852626,8.169096
10996,1.669881e+09,128.911388,34.853519,8.151822
10997,1.669881e+09,128.911509,34.854413,8.134548
10998,1.669882e+09,128.911629,34.855306,8.117274


In [200]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

## 저속 (7knot 미만 제거)

### segment

In [201]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [202]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [203]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

11.52599108138233
11.19293964068251
10.876903170636188
10.958129564553111
10.834623377860574
10.645074699369948
10.795498105301844
10.725744197270933
10.608700143738117
10.434356326496903
9.617231777573448
11.496648910640907
10.892665165055211
9.109985365251978
8.15215643591141
7.095096137994585
5.157831703524991
5.447660346146239
6.71280035575544
7.8881746587562205


### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [204]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [205]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_{interpolation_number}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [206]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 03:53:08.744564
평균: 0 days 01:37:32.944082910


In [207]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
81,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,121.6065,38.2938,10.0,102.0,2022-11-03 05:20:40.775978,2022-11-02 20:16:25,2022-11-02 07:44:12,under way using engine,741.880266,1245.460598,2022-11-05 01:35:16.201535,2022-11-05 01:34:31,0 days 00:00:45.201535
262,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9255,34.2735,7.4,60.0,2022-11-05 01:31:02.428314,2022-11-04 16:30:34,2022-11-04 04:14:10,under way using engine,112.615917,205.608677,2022-11-05 01:35:37.427207,2022-11-05 01:34:31,0 days 00:01:06.427207
36,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.9483,38.6342,11.1,94.0,2022-11-02 22:01:34.353827,2022-11-02 13:00:34,2022-11-02 12:56:10,under way using engine,888.189640,1505.410268,2022-11-05 01:37:04.486823,2022-11-05 01:34:31,0 days 00:02:33.486823
60,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,120.8479,38.5566,11.5,91.0,2022-11-03 01:43:38.286928,2022-11-02 16:43:24,2022-11-02 04:38:10,under way using engine,814.007071,1391.457653,2022-11-05 01:37:44.274180,2022-11-05 01:34:31,0 days 00:03:13.274180
261,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9146,34.2690,7.3,59.0,2022-11-05 01:25:59.438198,2022-11-04 16:25:43,2022-11-04 04:14:10,under way using engine,113.686369,209.015615,2022-11-05 01:30:46.427207,2022-11-05 01:34:31,0 days 00:03:44.572793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.9918,34.0627,8.5,88.0,2022-11-04 12:03:28.162145,2022-11-04 02:59:03,2022-11-02 07:44:12,under way using engine,277.992634,479.509365,2022-11-04 21:46:23.367199,2022-11-05 01:34:31,0 days 03:48:07.632801
104,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4097,35.1087,12.4,145.0,2022-11-04 05:19:43.093566,2022-11-03 20:19:06,2022-11-02 07:44:12,under way using engine,310.800535,545.995500,2022-11-04 21:46:12.255436,2022-11-05 01:34:31,0 days 03:48:18.744564
140,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.5674,34.4923,16.5,175.0,2022-11-04 08:21:23.318549,2022-11-03 23:21:07,2022-11-02 07:44:12,under way using engine,301.872964,659.184072,2022-11-04 21:45:00.885042,2022-11-05 01:34:31,0 days 03:49:30.114958
150,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.9837,34.0632,8.4,88.0,2022-11-04 11:58:25.435480,2022-11-04 02:56:13,2022-11-02 07:44:12,under way using engine,278.663426,482.472950,2022-11-04 21:43:33.367199,2022-11-05 01:34:31,0 days 03:50:57.632801


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [208]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [209]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_{interpolation_number}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [210]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 04:01:00.918432
평균: 0 days 01:36:03.689561170


In [211]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
81,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,121.6065,38.2938,10.0,102.0,2022-11-03 05:20:40.775978,2022-11-02 20:16:25,2022-11-02 07:44:12,under way using engine,741.880266,1245.460598,2022-11-05 01:35:16.201535,2022-11-05 01:34:31,0 days 00:00:45.201535
262,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9255,34.2735,7.4,60.0,2022-11-05 01:31:02.428314,2022-11-04 16:30:34,2022-11-04 04:14:10,under way using engine,112.615917,205.608677,2022-11-05 01:35:37.427207,2022-11-05 01:34:31,0 days 00:01:06.427207
235,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6274,34.1819,6.7,64.0,2022-11-04 23:14:46.584631,2022-11-04 14:14:22,2022-11-04 01:38:11,under way using engine,139.937034,270.161194,2022-11-05 01:35:38.081568,2022-11-05 01:34:31,0 days 00:01:07.081568
180,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.0338,34.0775,6.5,79.0,2022-11-04 18:37:09.715818,2022-11-04 09:36:22,2022-11-02 07:44:12,under way using engine,191.309380,376.465514,2022-11-05 01:36:08.720197,2022-11-05 01:34:31,0 days 00:01:37.720197
36,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.9483,38.6342,11.1,94.0,2022-11-02 22:01:34.353827,2022-11-02 13:00:34,2022-11-02 12:56:10,under way using engine,888.189640,1505.410268,2022-11-05 01:37:04.486823,2022-11-05 01:34:31,0 days 00:02:33.486823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.9837,34.0632,8.4,88.0,2022-11-04 11:58:25.435480,2022-11-04 02:56:13,2022-11-02 07:44:12,under way using engine,278.663426,482.472950,2022-11-04 21:43:33.367199,2022-11-05 01:34:31,0 days 03:50:57.632801
189,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.1331,34.0882,6.4,76.0,2022-11-04 19:22:35.135832,2022-11-04 10:22:05,2022-11-02 07:44:12,under way using engine,182.943349,363.661714,2022-11-04 21:43:21.081568,2022-11-05 01:34:31,0 days 03:51:09.918432
103,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4000,35.1233,12.6,150.0,2022-11-04 05:14:38.376056,2022-11-03 20:14:16,2022-11-02 07:44:12,under way using engine,311.739382,551.733145,2022-11-04 21:41:22.255436,2022-11-05 01:34:31,0 days 03:53:08.744564
188,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.1193,34.0860,6.4,76.0,2022-11-04 19:17:32.215013,2022-11-04 10:15:34,2022-11-02 07:44:12,under way using engine,184.144127,366.048666,2022-11-04 21:36:50.081568,2022-11-05 01:34:31,0 days 03:57:40.918432


# ETA 실험 (interpolation 12000), AKA BHUM 7항차 저속

In [212]:
interpolation_number = 12000

In [213]:
low_speed_interpolated_df = pd.read_csv(f"실험/upper_china_aka_bhum_low_speed_below_3_knot_{interpolation_number}_concaternated_df.csv")
low_speed_interpolated_df

,timestamp,longitude,latitude,speed
0,1.669615e+09,117.794500,38.979000,7.700000
1,1.669615e+09,117.795055,38.978604,7.766081
2,1.669615e+09,117.795610,38.978208,7.832161
3,1.669615e+09,117.796165,38.977813,7.898242
4,1.669616e+09,117.796720,38.977417,7.964323
...,...,...,...,...
11995,1.669881e+09,128.911307,34.852923,8.163338
11996,1.669881e+09,128.911418,34.853743,8.147503
11997,1.669881e+09,128.911529,34.854562,8.131669
11998,1.669882e+09,128.911639,34.855381,8.115834


In [214]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

## 저속 (7knot 미만 제거)

### segment

In [215]:
seg_length = int(len(low_speed_interpolated_df) / the_number_of_segment)

In [216]:
low_speed_df_list = []
for i in range(the_number_of_segment):
    idx = seg_length * (i + 1)
    if i == 0:
        df = low_speed_interpolated_df.loc[0:idx - 1]
        df.reset_index(inplace=True)
    else:
        df = low_speed_interpolated_df.loc[idx - seg_length:idx - 1]
        df.reset_index(inplace=True)
    low_speed_df_list.append(df)

In [217]:
for df in low_speed_df_list:
    d = df["speed"].mean()
    print(d)

11.526222805369029
11.192935982932905
10.876903360784514
10.958143083453997
10.83458812662909
10.64507824703861
10.795499916804353
10.725741980342757
10.608698813581205
10.434355644129711
9.617229471078149
11.49663810596172
10.89269026058943
9.109987915522543
8.152218128817722
7.095202893286487
5.157820103416513
5.4476510139557925
6.712656062127324
7.8881670363934395


### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [218]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [219]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
    if current_speed < 7:
        continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.1.2.3_{interpolation_number}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [220]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 03:52:57.767473
평균: 0 days 01:38:12.224925502


In [221]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
81,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,121.6065,38.2938,10.0,102.0,2022-11-03 05:20:40.775978,2022-11-02 20:16:25,2022-11-02 07:44:12,under way using engine,741.880266,1245.460598,2022-11-05 01:35:36.137215,2022-11-05 01:34:31,0 days 00:01:05.137215
262,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9255,34.2735,7.4,60.0,2022-11-05 01:31:02.428314,2022-11-04 16:30:34,2022-11-04 04:14:10,under way using engine,112.615917,205.608677,2022-11-05 01:35:41.809449,2022-11-05 01:34:31,0 days 00:01:10.809449
36,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.9483,38.6342,11.1,94.0,2022-11-02 22:01:34.353827,2022-11-02 13:00:34,2022-11-02 12:56:10,under way using engine,888.189640,1505.410268,2022-11-05 01:37:28.192728,2022-11-05 01:34:31,0 days 00:02:57.192728
60,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,120.8479,38.5566,11.5,91.0,2022-11-03 01:43:38.286928,2022-11-02 16:43:24,2022-11-02 04:38:10,under way using engine,814.007071,1391.457653,2022-11-05 01:38:05.981601,2022-11-05 01:34:31,0 days 00:03:34.981601
261,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9146,34.2690,7.3,59.0,2022-11-05 01:25:59.438198,2022-11-04 16:25:43,2022-11-04 04:14:10,under way using engine,113.686369,209.015615,2022-11-05 01:30:50.809449,2022-11-05 01:34:31,0 days 00:03:40.190551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.9918,34.0627,8.5,88.0,2022-11-04 12:03:28.162145,2022-11-04 02:59:03,2022-11-02 07:44:12,under way using engine,277.992634,479.509365,2022-11-04 21:46:31.340549,2022-11-05 01:34:31,0 days 03:47:59.659451
104,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4097,35.1087,12.4,145.0,2022-11-04 05:19:43.093566,2022-11-03 20:19:06,2022-11-02 07:44:12,under way using engine,310.800535,545.995500,2022-11-04 21:46:23.232527,2022-11-05 01:34:31,0 days 03:48:07.767473
140,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.5674,34.4923,16.5,175.0,2022-11-04 08:21:23.318549,2022-11-03 23:21:07,2022-11-02 07:44:12,under way using engine,301.872964,659.184072,2022-11-04 21:45:10.511547,2022-11-05 01:34:31,0 days 03:49:20.488453
150,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.9837,34.0632,8.4,88.0,2022-11-04 11:58:25.435480,2022-11-04 02:56:13,2022-11-02 07:44:12,under way using engine,278.663426,482.472950,2022-11-04 21:43:41.340549,2022-11-05 01:34:31,0 days 03:50:49.659451


## 저속 (7knot 미만 포함)

### segment로 평균 속도 적용 ETA 계산

#### 3. 순간 속력 이용 + 알고리즘 x  +  감속 계수 x

In [222]:
test_df_26 = pd.read_csv("voy_path/upper_china/processed/AKA_BHUM_9V7282_2022_7processed.csv")
test_df_26['timestamp'] = pd.to_datetime(test_df_26['timestamp'])
test_df_26['time_position_data_received'] = pd.to_datetime(test_df_26['time_position_data_received'])
test_df_26['time_voyage_data_received'] = pd.to_datetime(test_df_26['time_voyage_data_received'])
test_df_26.drop('index', axis=1, inplace=True)
test_df_26 = test_df_26.loc[test_df_26[['longitude','latitude','time_position_data_received']].drop_duplicates().index]
test_df_26.reset_index(inplace=True, drop=True)

In [223]:
for col in test_df_26.index:
    if col == test_df_26.index[-8]:
        break
    # 테스트 데이터
    re_sample = test_df_26.loc[col,:]
    longitude = re_sample["longitude"]
    latitude = re_sample["latitude"]
    current_point = (latitude, longitude)
    
    # 1. 순간 속력 (AIS) 
    current_speed = re_sample["speed"]
    current_position_timestamp = re_sample["time_position_data_received"]
#     if current_speed < 7:
#         continue
    # 현재 위치에서 가장 짧은 segment 찾기
    shortest_distance = 9999
    for seg_no, seg_df in enumerate(low_speed_df_list):
        for idx in seg_df.index:
            LONGITUDE = seg_df.loc[idx, "longitude"]
            LATITUDE = seg_df.loc[idx, "latitude"]

            ais_point = (LATITUDE, LONGITUDE)
            distance = haversine(current_point, ais_point, unit="km")
            # 가장 짧은 segment 간의 거리, segment 번호, 인덱스 저장
            if distance <= shortest_distance:
                shortest_distance = distance 
                shortest_point_idx = seg_df.loc[idx, "index"]
                shortest_seg_no = seg_no
                
    # 다음 인덱스
#     optimal_point_idx = optimal_next_point(current_point, shortest_point_idx, shortest_distance, low_speed_interpolated_df)
    optimal_point_idx = low_speed_interpolated_df.query(f'index == {shortest_point_idx}').index[0] + 1
    lon_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "longitude"]
    lat_optimal = low_speed_interpolated_df.loc[optimal_point_idx, "latitude"]
    
    optimal_point = (lat_optimal, lon_optimal)
    optimal_distance = haversine(current_point, optimal_point, unit="km")
    km_h = current_speed * 1.852
    init_duration = cal_duration(km_h, optimal_distance)
    
    
    # 가장 가까운 segment 찾기
    
    optimal_seg_no = find_segment_no(optimal_point_idx, low_speed_interpolated_df,the_number_of_segment)
    if optimal_seg_no == the_number_of_segment:
        continue
    initial_seg_df = low_speed_df_list[optimal_seg_no]
    # speed_damping_ratio는 각 segment에 맞게
#     speed_damping_ratio = current_speed / initial_seg_df["speed"].mean()
    speed_damping_ratio = 1
    km_h = current_speed * 1.852 * speed_damping_ratio
    
    
    remaining_initial_seg_df = initial_seg_df.loc[optimal_point_idx:,:]
    remaining_initial_seg_df.reset_index(inplace=True, drop=True)
    accu_distance = 0
    for idx in remaining_initial_seg_df.index:
        if idx != 0:
            LONGITUDE = remaining_initial_seg_df.loc[idx, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = remaining_initial_seg_df.loc[idx - 1, "longitude"]
            LATITUDE = remaining_initial_seg_df.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)
            
            accu_distance += haversine(current_coord, previous_current_coord, unit="km")
            
    duration_0 = cal_duration(km_h, accu_distance)
    duration_0 = duration_0 + init_duration
    
    
        
    dura_list = []  
    for next_seg_no in range(optimal_seg_no + 1, the_number_of_segment):
        next_seg_df = low_speed_df_list[next_seg_no]
        
        seg_mean_speed = next_seg_df['speed'].mean()
#         speed_damping_ratio = current_speed / seg_mean_speed
        speed_damping_ratio = 1
        damped_mean_speed = seg_mean_speed * speed_damping_ratio
        km_h = damped_mean_speed * 1.852
        distance = reckon_point_lengths(next_seg_df)
        dura_list.append(cal_duration(km_h, distance))
    
    accu_timedelta = datetime.timedelta(seconds=0)
    for i in dura_list:
        accu_timedelta += i
    # 현재 시간 + duration = ETA
    ETA = current_position_timestamp + accu_timedelta
    test_df_26.loc[col,'ETA'] = ETA
    optimal_seg_no = 0

test_df_26['ATA'] = test_df_26.loc[test_df_26.index[-1],"time_position_data_received"]
test_df_26['error'] = abs(test_df_26['ATA'] - test_df_26['ETA'])

experiment_no = f"4.2.1.3_{interpolation_number}".replace(".","_")
result_df_dict['experiment_no'].append(experiment_no)
result_df_dict['mean_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].mean())
result_df_dict['max_speed'].append(test_df_26.loc[test_df_26.index[:-20]]["error"].max())

globals()[f"exp_result_df_{experiment_no}"] = test_df_26

In [224]:
print("최대:", test_df_26.loc[test_df_26.index[:-20]]["error"].max())
print("평균:", test_df_26.loc[test_df_26.index[:-20]]["error"].mean())

최대: 0 days 04:00:55.357673
평균: 0 days 01:36:28.199072811


In [225]:
test_df_26.loc[test_df_26['error'].dropna(axis=0).index].sort_values(by="error")

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption,ETA,ATA,error
81,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,121.6065,38.2938,10.0,102.0,2022-11-03 05:20:40.775978,2022-11-02 20:16:25,2022-11-02 07:44:12,under way using engine,741.880266,1245.460598,2022-11-05 01:35:36.137215,2022-11-05 01:34:31,0 days 00:01:05.137215
262,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.9255,34.2735,7.4,60.0,2022-11-05 01:31:02.428314,2022-11-04 16:30:34,2022-11-04 04:14:10,under way using engine,112.615917,205.608677,2022-11-05 01:35:41.809449,2022-11-05 01:34:31,0 days 00:01:10.809449
235,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.6274,34.1819,6.7,64.0,2022-11-04 23:14:46.584631,2022-11-04 14:14:22,2022-11-04 01:38:11,under way using engine,139.937034,270.161194,2022-11-05 01:35:43.642327,2022-11-05 01:34:31,0 days 00:01:12.642327
180,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.0338,34.0775,6.5,79.0,2022-11-04 18:37:09.715818,2022-11-04 09:36:22,2022-11-02 07:44:12,under way using engine,191.309380,376.465514,2022-11-05 01:36:15.001416,2022-11-05 01:34:31,0 days 00:01:44.001416
36,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,119.9483,38.6342,11.1,94.0,2022-11-02 22:01:34.353827,2022-11-02 13:00:34,2022-11-02 12:56:10,under way using engine,888.189640,1505.410268,2022-11-05 01:37:28.192728,2022-11-05 01:34:31,0 days 00:02:57.192728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.9837,34.0632,8.4,88.0,2022-11-04 11:58:25.435480,2022-11-04 02:56:13,2022-11-02 07:44:12,under way using engine,278.663426,482.472950,2022-11-04 21:43:41.340549,2022-11-05 01:34:31,0 days 03:50:49.659451
189,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.1331,34.0882,6.4,76.0,2022-11-04 19:22:35.135832,2022-11-04 10:22:05,2022-11-02 07:44:12,under way using engine,182.943349,363.661714,2022-11-04 21:43:26.642327,2022-11-05 01:34:31,0 days 03:51:04.357673
103,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,125.4000,35.1233,12.6,150.0,2022-11-04 05:14:38.376056,2022-11-03 20:14:16,2022-11-02 07:44:12,under way using engine,311.739382,551.733145,2022-11-04 21:41:33.232527,2022-11-05 01:34:31,0 days 03:52:57.767473
188,AKA BHUM,563135800,9307011.0,5888,KRBNP,11050900,127.1193,34.0860,6.4,76.0,2022-11-04 19:17:32.215013,2022-11-04 10:15:34,2022-11-02 07:44:12,under way using engine,184.144127,366.048666,2022-11-04 21:36:55.642327,2022-11-05 01:34:31,0 days 03:57:35.357673


# 결과 확인

In [226]:
result_pool = pd.DataFrame(result_df_dict)
result_pool

,experiment_no,mean_speed,max_speed
0,4_1_2_3_1000,0 days 01:56:47.617395169,0 days 04:51:19.179098
1,4_2_1_3_1000,0 days 01:53:11.506819881,0 days 04:51:19.179098
2,4_1_2_3_2000,0 days 01:40:27.041994539,0 days 04:13:16.915045
3,4_2_1_3_2000,0 days 01:39:28.202920130,0 days 04:13:16.915045
4,4_1_2_3_2500,0 days 01:38:28.728328450,0 days 04:06:05.925523
5,4_2_1_3_2500,0 days 01:37:58.273564750,0 days 04:06:05.925523
6,4_1_2_3_3000,0 days 01:37:52.042216591,0 days 04:04:27.803001
7,4_2_1_3_3000,0 days 01:37:25.740725841,0 days 04:04:27.803001
8,4_1_2_3_4000,0 days 01:37:21.127528056,0 days 04:02:23.303773
9,4_2_1_3_4000,0 days 01:36:54.815113510,0 days 04:02:54.965135


In [231]:
result_pool.sort_values(by="mean_speed")

,experiment_no,mean_speed,max_speed
11,4_2_1_3_5000,0 days 01:35:57.489220507,0 days 04:02:19.904080
13,4_2_1_3_7500,0 days 01:35:59.055857644,0 days 04:01:31.677562
19,4_2_1_3_11000,0 days 01:36:03.689561170,0 days 04:01:00.918432
21,4_2_1_3_12000,0 days 01:36:28.199072811,0 days 04:00:55.357673
17,4_2_1_3_10000,0 days 01:36:43.464540446,0 days 04:01:07.590395
15,4_2_1_3_9000,0 days 01:36:46.320734629,0 days 04:01:15.220200
9,4_2_1_3_4000,0 days 01:36:54.815113510,0 days 04:02:54.965135
10,4_1_2_3_5000,0 days 01:37:03.167174211,0 days 04:01:10.044358
12,4_1_2_3_7500,0 days 01:37:17.984879253,0 days 03:59:31.604528
8,4_1_2_3_4000,0 days 01:37:21.127528056,0 days 04:02:23.303773


In [228]:
for i in globals():
    if 'exp_result_df_' in i:
        globals()[i]

In [229]:
# exp_result_df_4_1_2_3

In [230]:
# result_pool.to_csv(f"230126_result_pool_{interpolation_number}.csv", encoding="utf-8 sig", header=True, index=False)